<a href="https://colab.research.google.com/github/FedorovIV1978/MY_new_rep/blob/main/%D1%80%D0%B0%D0%B1%D0%BE%D1%87%D0%B0%D1%8F_%D0%B2%D0%B5%D1%80%D1%81%D0%B8%D1%8F_%D1%81_%D0%BE%D1%82%D0%BF%D1%80%D0%B0%D0%B2%D0%BA%D0%BE%D0%B9_%D1%84%D0%BE%D1%82%D0%BE_%D1%82%D0%BE%D0%B2%D0%B0%D1%80%D0%B0_%22%D0%9E%D0%9E%D0%9F_%D0%91%D0%BE%D1%82%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#блок для установки необходимых библиотек
!pip install telegram
!pip install python-telegram-bot
#!pip install -U qdrant-client
!pip install -U sentence-transformers
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

  Preparing metadata (setup.py) ... done
  Created wheel for telegram: filename=telegram-0.0.1-py3-none-any.whl size=1293 sha256=8784df073b75dc797249cf99fb36d6e3bd05548b545fa80a4db06ad34eabafea
  Stored in directory: /root/.cache/pip/wheels/1e/b7/10/7a0e3b8f8c01e0d9d8a1e9913251932ca9f0a394e98ade3056
Successfully built telegram
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 669.5/669.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.9/275.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
#блок импорта необходимых библиотек
import nest_asyncio
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Application, CommandHandler, CallbackQueryHandler, ContextTypes, MessageHandler, filters
import re
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
from openai import OpenAI
import openai
from sentence_transformers import SentenceTransformer, util
#from qdrant_client import models, QdrantClient
import os
import pickle

In [47]:
# блок объявления переменных
history=[]
PROXY_API_KEY ='ваш ключ'
client = OpenAI(
    api_key=f"{PROXY_API_KEY}",
    base_url="https://api.proxyapi.ru/openai/v1",)
model="gpt-4o-mini"
max_tokens=300
temperature=0.7
#user_response_data=[]
url_site='https://ru.siberianhealth.com/ru/shop/catalog/product/'
# переменная словарь headers для имитации пользовательского запроса при парсинге
headers = {
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
"Accept-Language": "en-US,en;q=0.5",
"Connection": "keep-alive"}
# переменная словарь со списком категорий
category_urls = {
    "Витамины и добавки": 'https://ru.siberianhealth.com/ru/shop/catalog/vitaminy-i-dobavki-1/',
    "Спорт": 'https://ru.siberianhealth.com/ru/shop/catalog/sport-3/',
    "Питание": 'https://ru.siberianhealth.com/ru/shop/catalog/pitanie-2/',
    "Красота": 'https://ru.siberianhealth.com/ru/shop/catalog/krasota-4/',
    "Парфюмерия": 'https://ru.siberianhealth.com/ru/shop/catalog/parfyumeriya-212/',
    "Новинки": 'https://ru.siberianhealth.com/ru/shop/new/'
}

company ="""Siberian Wellness – для тех, кто хочет жить ярко!
Честный бизнес для wellness-предпринимателей: международные возможности, ответственный экоподход, глобальное производство инновационных продуктов для здоровья, спорта и красоты.

28
лет стабильного бизнеса от Азии до США
60+
стран, где можно купить нашу продукцию
300+
уникальных формул для здоровья, красоты и активной жизни
50+
наград за товары
blot-left
Наши ценности мы обрели благодаря сибирским корням: Siberian Wellness была основана там, где уважают традиции, развивают науку, берегут и ценят природу. Сегодня натуральные продукты
 для красоты, здоровья и спорта, а также эксклюзивная парфюмерия под всеми нашими брендами пользуются спросом в 60+ странах мира. Процесс их создания тоже стал международным:
  производственные комплексы Siberian Wellness работают в России, Сербии, Италии, Узбекистане, и мы продолжаем развитие. Компания стремится к тому, чтобы людям в любой точке
   мира была доступна как эффективность универсального ассортимента Siberian Wellness, так и созданные с учетом особенностей каждого из регионов специализированные линейки продуктов.
    В каждой локации мы берем максимум от достижений науки и богатства местной природы и отдаем это вам – в виде инновационных решений, расширения горизонта возможностей, заботы
     о нашей общей планете. И главное достижение, которым мы можем гордиться, – доверие Бизнес-Партнеров и сотен тысяч Клиентов."""

# переменная с разметкой категорий и подкатегорий для адресного ориентирования пользователя в списке товаров,
# бот ступенчато подходит к интересующему пользователя товару
category_article= {'витамины и добавки':{'иммунитет и защита организма':{'сильный иммунитет': ['500927', '501072', '500940', '500715', '500627', '500526', '500094', '500048',
                                                                                               '400273', '400271', '501590', '501329', '500918', '500440'],
                                                                      'антиоксиданты': [ '501420', '501352', '501233', '501236', '500954', '500662', '500652', '500630'],
                                                                      'антипаразитарная защита': ['500634', '500586', '500581', '500116', '501592'],
                                                                      'профилактика аллергии' : ['501001', '500663', '500659', '400237', '501640'],
                                                                      'профилактика диабета': ['501081', '500952', '500632', '500589', '500543']},
                                         'поддержка органов и систем':  {'защита печени':['501439', '501418', '501188', '500943', '500213', '500130', '400268', '501078'],
                                                                         'защита почек': ['501234', '501200', '500656', '500631', '400753', '400274', '400266'],
                                                                         'защита сердца': ['501429', '501325', '500689', '500661', '500629',
                                                                                           '500443', '500072', '501597'],
                                                                         'защита сосудов': ['501419', '500625', '500577', '400270', '400267'],
                                                                         'поддержка жкт': ['501546', '501545', '501544', '501335', '501400', '500660', '500633', '400265'],
                                                                         'микрофлора кишечника': ['501404', '501462', '501466', '501464', '501501', '501141', '501143']},
                                         'здоровье суставов и костей':  {'суставы':['501227', '500587', '500570', '500065', '415535', '414328', '410054', '402692', '501079'],
                                                                         'поддержка костной ткани':['501505', '501084', '500914', '500820', '500676', '500651', '500628', '500467', '500284']},
                                         'когнитивное и психоэмоциональное здоровье': {'поддержка мозга':['501289', '501237', '500654', '500175', '500071', '501604', '500481'],
                                                                                       'успокоительные средства':['501291', '501431',  '501186', '500931', '500657', '500583', '400264', '501596', '501591']},
                                         'красота и здоровье женщин': {'антивозрастной':['501139', '501088', '500655', '500020', '501642', '501638', '501616',
                                                                                         '501184', '501041', '500658', '500572', '500537', '500469', '500172'],
                                                                       'красота и здоровье женщин':['501569', '501407', '501405', '501331', '501330', '501290','501251', '501187', '500864', '500650',
                                                                                                    '501593', '500362', '500571']},
                                         'общее здоровье и профилактика':{'тонизирующие комплексы':['501183', '500327', '501637'],
                                                                          'антигеморроидальные средства':['501094', '500626'],
                                                                          'защита зрения':['501182', '501185', '500688', '500653', '500361', '500050', '501080']}},
                   'для спорта':{'для спорта':{'протеины':['501457', '501456', '501455', '108221', '501388', '501389', '108310'],
                                               'аминокислоты':['501567', '501217', '501092', '500285', '500277'],
                                               'витамины и микроэлементы':['501589', '500284'],
                                               'продукты для суставов':['501493', '501031', '500276'],
                                               'коллаген':['501181', '501152', '501663'],
                                               'кремы бальзамы':['418681', '414328', '410054'],
                                               'стройная фигура': ['501236', '500874', '500968', '500967', '500966', '500964']}},
                   'здоровое питание':{'здоровое питание':{'мармелад':['501335', '501065', '500563', '501063', '500960', '500959', '500514'],
                                                           'драже леденцы':['501305'],
                                                           'вегетарианское питание':['501457', '501456', '500940', '500713', '501338'],
                                                           'функциональные напитки':['501434', '501430', '501061', '501444', '500958', '500957'],
                                                           'батончики':['501452', '501441', '500644', '500568', '500566', '500490'],
                                                           'заменители питания':['501388', '501389', '501390', '501337', '501336'],
                                                           'медовые пасты':['500941', '500940'],
                                                           'хрустящие шарики':['501566',  '500670', '501329']}},
                   'для поддержания красоты': {'уход за лицом':{'очищение':['427182', '427511', '427518', '420832', '424403', '420833', '418442', '418441', '422377', '414339', '418443', '422644', '418896'],
                                                                'тонизирование':['427510', '423110', '420834', '422394', '422447', '418450', '418451', '411582'],
                                                                'маски пилинги':['427517', '420839', '424406', '418444', '422448', '418449', '422477', '420757', '420758', '415754', '415753'],
                                                                'сыворотки':['425760', '425761', '425762', '421695', '420840', '424404', '424750', '420841', '418895', '427637', '420838',
                                                                             '418452', '425531', '416108', '416106', '416105'],
                                                                'кремы':['427639', '427638', '427641', '427640', '427513', '427509', '427508', '427514', '427512', '427515', '422670', '427516', '422449',
                                                                         '418453', '425080', '424405', '420862', '420861', '418446', '418445', '422645', '414563',
                                                                         '411584', '411583', '411211', '404778', '404777'],
                                                                'уход за кожей вокруг глаз':['422671', '427535', '420863', '418448', '411212', '410682', '422646'],
                                                                'защита от солнца':['407583', '424161'],
                                                                'другие товары':['415504']},
                                               'уход за волосами':{'шампуни':['427524', '424410', '424442', '424439', '424434', '424437'],
                                                                   'кондиционеры':['424441', '424412', '424438', '424440', '424435'],
                                                                   'маски и спреи':['427523', '427522', '424436', '424441', '424380', '427531', '424443', '422364']},
                                               'уход за телом':{'для душа и ванны':['424407', '422290', '425701', '419577', '427624', '427561'],
                                                                'скрабы для тела':['427529', '427580', '423113'],
                                                                'уход за руками':['427526', '426347', '427528', '427527', '427532', '427749', '427748', '427747', '427746', '426348', '424685', '421684',
                                                                                  '421685', '421686'],
                                                                'уход за ногами':['426349', '427530', '416559'],
                                                                'интимная гигиена':['427504', '423394', '427503', '427502', '424752', '424751', '425705', '423400', '411568'],
                                                                'дезодоранты':['427521', '427520', '427519', '423959', '406402'],
                                                                'кремы':['421691', '422291', '425702', '424408', '425687', '422395', '418447', '411382', '427787'],
                                                                'бальзамы и масла':['427525', '424836', '417706', '416558', '415535', '414328', '410054', '409066',
                                                                                    '402860', '402692', '427560', '422288', '422287', '422289'],
                                                                'защита от солнца':['427534', '423888', '423886']},
                                               'уход за полостью рта':{'зубные пасты':['427788', '424159', '421916', '417382', '417381', '417379', '427801', '411381', '411376'],
                                                                       'спреи и ополаскиватели':['409069'],
                                                                       'зубные щетки':['425017']},
                                               'макияж':{'тонирующие средства':['411998'],
                                                         'помада и блески для губ':['419767', '419766', '416776', '416775', '416773', '412668', '407731'],
                                                         'тушь':['416095', '407676'],
                                                         'тени и подводки':['412663', '412662']},
                                               'мужчинам':{'бритье и уход за кожей':['427506', '427507', '419582', '419583'],
                                                           'тело и волосы':['427562', '427505', '419577', '427561', '423958']},
                                               'детская косметика':{'детская косметика':['425758', '426985', '425687']}},
                   'парфюмерия':{'парфюмерия':{'для женщин':[ '427650', '427649', '108233', '108235', '108236', '108237', '108232', '425181', '108195', '423514',
                                                             '423513', '422278', '424201', '108181', '108184', '108163', '422277', '424937'],
                                               'для мужчин':[ '108212', '427653', '427652', '427651', '424137', '108182', '108180', '108179',
                                                             '108178', '422772', '411164', '419969', '417417', '411162', '108189', '108188'],
                                               'унисекс':['425139', '425137', '108171', '108173', '108174', '108231', '422279', '424444', '422774', '418434'],
                                               'парфюмированный уход за телом':['427745', '427749', '427748', '427744'],
                                               'цветочные ароматы':['425177', '108193', '108169', '108168', '424134', '423516', '423510', '424152', '424148', '424143'],
                                               'цитрусовые ароматы':['108200', '425248', '425246',  '424203', '108167', '424131', '108185', '108183','422771', '422766', '417419', '425003'],
                                               'древесные ароматы':['425244', '427001', '424198', '424941', '425403', '419968', '419967', '417626', '411162', '108189', '428596', '108164', ],
                                               'фруктовые ароматы':[ '425175', '108201', '425250', '108213', '427000', '423516', '423515', '426148', '423993', '108188', '108192', '108214'],
                                               'восточные ароматы':['108177', '108172', '424202', '422280', '425006', '422281', '424200', '419968', '419967', '417626', '417418'],
                                               'дневные ароматы':[ '425175', '108199', '108200', '108201', '424131', '108177', '424199', '423516', '423512', '424150', '424146'],
                                               'вечерние ароматы':['425175', '108183', '424941', '426148', '425006', '424200', '419968', '419967', '417626', '417418', '411162', '428596'],
                                               'подарочные наборы':['427757', '425006', '425005', '427758', '425004', '425003'],
                                               'миниатюры ароматов':['108234', '108175', '424949', '422282', '423511', '424153', '424151', '424149', '424147', '424145']}},
                   'чаи и напитки':{'чаи и напитки':{'чаи и напитки':['500589', '500588', '500587', '500586', '500585', '500584', '500583', '500582', '500581', '501592', '501591']}},
                   'бальзамы':{'бальзамы':{'бальзамы':['424836', '418681', '417706', '416559', '416558', '415535', '414328', '410054', '409066', '402860', '402692', '409069']}},
                   'сопутствующие товары':{'сопутствующие товары':{'атрибутика':['107974'],
                                                                   'аксессуары':['108160', '108161', '107974', '107631', '108290', '108310', '108265', '107620', '107557', '107307', '107283', '106736'],
                                                                   'каталоги':['108318', '108300', '108328', '108309', '108295', '108274'],
                                                                   'книги':['414466', '414465', '107418', '107317', '106978', '105524'],
                                                                   'сумки и пакеты':['108325', '108326', '108242', '108241', '107963', '107974', '800928', '107630', '107525', '107524', '107463',
                                                                                     '107385', '107382', '107381', '107380', '107313', '106664']}}}
# данная переменная используеться в анализаторе ответа от пользователя, через нее модель определяет наиболее подходящий вариант предоставляемых к openAi данных
# class Response_user_text сравнивает с этой переменной ответ пользователя, выделяя чистую сущьность, class Logicif определяет категорию ответа на основании чего
# подготавливаются необходимые данные

logiс_data={1: ['витамины и добавки','для поддержания красоты'],
            2: ['иммунитет и защита организма','поддержка органов и систем','здоровье суставов и костей','когнитивное и психоэмоциональное здоровье','красота и здоровье женщин',
                'общее здоровье и профилактика','для спорта','здоровое питание','уход за лицом','уход за волосами','уход за телом','уход за полостью рта','макияж','мужчинам',
                'парфюмерия','сопутствующие товары'],
            3:['сильный иммунитет','антиоксиданты','антипаразитарная защита','профилактика аллергии','профилактика диабета','защита печени','защита почек','защита сердца',
               'защита сосудов','поддержка жкт','микрофлора кишечника','суставы','поддержка костной ткани','поддержка мозга','успокоительные средства','антивозрастной','красота и здоровье женщин',
               'тонизирующие комплексы', 'антигеморроидальные средства','защита зрения','протеины','аминокислоты','витамины и микроэлементы','продукты для суставов','коллаген','кремы бальзамы',
               'стройная фигура','мармелад','драже леденцы','вегетарианское питание','функциональные напитки','батончики','заменители питания','медовые пасты','хрустящие шарики','очищение',
               'тонизирование','маски пилинги','сыворотки','кремы','уход за кожей вокруг глаз','защита от солнца','другие товары','шампуни','кондиционеры','маски и спреи','для душа и ванны',
               'скрабы для тела','уход за руками','уход за ногами','интимная гигиена','дезодоранты','кремы','бальзамы и масла','защита от солнца','зубные пасты','спреи и ополаскиватели',
               'зубные щетки','тонирующие средства','помада и блески для губ','тушь','тени и подводки','брови','бритье и уход за кожей','тело и волосы','детская косметика','лля женщин'
               'для мужчин','унисекс','парфюмированный уход за телом','цветочные ароматы','цитрусовые ароматы','древесные ароматы','вечерние ароматы','подарочные наборы',
               'миниатюры ароматов','чаи и напитки','бальзамы','атрибутика','аксессуары','каталоги','книги','сумки и пакеты'],
            4:['хотел бы посмотреть что то еще','дальше','далее','что то еще','другой вариант','какие то другие товары','другой товар','что можно посмотреть еще','по другому товару','другая категория','это не заинтересовало'],
            5:['как приобрести','хотел бы купить','связаться с менеджером','сделать заказ','есть желание','желаю купить','нужен этот товар','готов приобрести','нужно заказать'],
            6:['что вы за компания','чем вы занимаетесь','для чего ваша продукция','кто вы такие','где ваше производство','расскажите о вашей компании','что вы можете рассказать о себе',
 'поделитесь информацией о вашей компании', 'какие факты о вас могли бы быть интересны', 'что вы можете рассказать о своей деятельности', 'расскажите о ваших достижениях', 'какова история вашей компании', 'что отличает вас от конкурентов',
 'какие услуги вы предоставляете', 'в чем заключается ваша миссия', 'каковы ваши основные ценности', 'что вдохновило вас на создание компании', 'как вы видите будущее вашей компании', 'какие у вас есть уникальные предложения',
 'как вы поддерживаете связь с клиентами', 'что вам нравится в вашей работе', 'какие проблемы вы решаете для клиентов', 'как вы оцениваете успех вашей компании', 'в чем заключается ваша экспертиза',
 'какова ваша целевая аудитория', 'какие технологии вы используете в работе', 'как вы справляетесь с вызовами в бизнесе', 'что вы можете рассказать о команде вашей компании', 'каковы ваши планы на ближайшее будущее',
 'в чем заключается ваш подход к инновациям', 'как вы обеспечиваете качество своих услуг', 'что делает вашу компанию уникальной на рынке', 'как вы измеряете удовлетворенность клиентов',
 'какие у вас есть партнеры или сотрудничества', 'что вы можете рассказать о вашем опыте работы в отрасли', 'каковы ваши конкурентные преимущества', 'в чем состоит ваша философия ведения бизнеса',
 'как вы обучаете своих сотрудников', 'что вы делаете для устойчивого развития'
],
            7:['какие товары у вас есть','что у вас за продукция','какие категории','что вы продаете','что вы предлааете', 'что вы можете предложить'],
            8:['Новомин-L',
 'Пробиотик Gyneflor boulardii для защиты женской мочеполовой системы и профилактики кишечных расстройств',
 'Комплекс растительных ферментов Plant Enzymes Complex',
 'Ферментный комплекс «Бетаин HCl + протеаза» для поддержки в переваривании и усвоении белка',
 '«Омега-3 Ультра» с витамином Е для поддержки организма',
 'Фермент «Лактаза» для усвоения лактозы и комфортного пищеварения',
 'Органический кремний',
 'Лецитин подсолнечника с МСТ-маслом и бета-каротином',
 'Вегетарианский комплекс Totally Vegan Omega-3',
 'Пробиотик Senior для поддержания иммунитета и защиты от вирусов',
 'Пробиотик Nutrition Comfort для комфортного пищеварения (3+)',
 'Пробиотик Women’s Balance для баланса женской интимной микрофлоры',
 'Мио-инозитол & D-хиро-инозитол',
 'Пустырник и мята',
 'Дикий ямс & Изофлавоны сои',
 '100% натуральных витаминов A, C, Е',
 'Омега-3 & Натуральный витамин К2 MenaQ7®',
 'Натуральный витамин E',
 'S-ацетил-L-глутатион',
 'Dino Vitamino, ягодный сироп с витаминами и минералами',
 'OMEGAlodon (классик), комплекс омега-3 кислот',
 'Метилфолат',
 'Turbo Tea (Очищающий турбочай)',
 'Истоки чистоты. Формула 3',
 'Гиалуроновая кислота и натуральный витамин С',
 'Immunotops, капсулы с витамином D3',
 'Коэнзим Q10',
 'Перуанская мака',
 'MetaBio-мармелад (кактус-инжир)',
 'Хронолонг',
 'Витамин К2',
 'Биодоступный куркумин',
 'Immunotops, хрустящие шарики с инулином (вишня)',
 'Хелат железа',
 'Лактоферрин',
 'Омега-3 Ультра',
 'Глицин',
 'Энергетический бустер',
 'Витамин С и рутин',
 'Альфа-липоевая кислота',
 'CoreNRG, антиоксидантный бустер',
 'Dino Vision, жевательные таблетки с черникой',
 'Age Тhеrару Antioxidants',
 'D-манноза & Северная клюква',
 'Dino Vitamino, сироп с витаминами и минералами',
 'Immunotops, фруктовые жевательные таблетки с витаминами A, C и D',
 'Dino Lactino, пектиновый сорбент',
 'Neurovision',
 'BeautySense',
 'Detox Pro.Active',
 'Stress Re.Live',
 'Dino Lactino, какао-напиток',
 'Венотоник с диосмином и полифенолами (пряная вишня)',
 'OMEGAlodon (мультифрукт), комплекс омега-3 кислот',
 'L-аргинин',
 'Ортилия и Зимолюбка',
 'Новомин',
 'Органический кальций',
 'Хромлипаза',
 'Метилсульфонилметан',
 'Очищающий фитосорбент Pure Life (саше)',
 'Органический германий',
 'GLUCO Box / Контроль уровня сахара',
 'Detox Pro.Active',
 'Antistress, медовая паста с мелиссой и пассифлорой для поддержки нервной системы',
 ' Immuno, медовая паста с душицей и чабрецом для укрепления иммунитета',
 'RELAX Box / Защита от стресса',
 'Органический кальций',
 'Витамин D3',
 'Сироп для иммунитета Immunotops без сахара',
 'Напиток Weight Control (малина-гранат)',
 'MAMA Box Грудное вскармливание',
 'MAMA Box Беременность',
 'Ликопин и омега-3',
 'Лютеин и зеаксантин',
 'Витамины с кальцием',
 'Calcitops, хрустящие шарики с какао-маслом (малина)',
 'Эльбифид',
 'Сибирский лен и омега-3',
 'Северная омега-3',
 'Натуральный бета-каротин и облепиха',
 'Бораго и амарант',
 'Валериана и мелисса',
 'Медвежьи ушки и брусника',
 'Йохимбе и сибирский женьшень',
 'Гинкго билоба и байкальский шлемник',
 'Арония и лютеин',
 'Витамины красоты',
 'Глюкозамин и хондроитин',
 'BeautySense',
 'Кишечный фитосорбент Intestinal Defense',
 'Суставной фитосорбент Joint Comfort',
 'Очищающий фитосорбент Pure Life',
 'Органический цинк',
 'Органический селен',
 'Органический магний',
 'Органический кальций',
 'Органическое железо',
 'Диосмин и рутин',
 'Бетаин и В-витамины',
 'Фиточай из диких трав № 9 (Углеводный контроль)',
 'Фиточай из диких трав № 8 (Сердечный комфорт)',
 'Фиточай из диких трав № 7 (Легкость движений)',
 'Фиточай из диких трав № 6 (Защита печени)',
 'Фиточай из диких трав № 5 (Комфортное пищеварение)',
 'Фиточай из диких трав № 4 (Легкое дыхание)',
 'Фиточай из диких трав № 3 (Природный антистресс)',
 'Фиточай из диких трав № 2 (Женская гармония)',
 'Фиточай из диких трав № 1 (Очищение и дренаж)',
 'Северная клюква и В-витамины',
 '3D Flex Cube',
 'Жевательный мармелад с прополисом',
 'Напиток Weight Control (яблоко-лимон)',
 'Immuno Box / Иммуно бокс',
 'Lite Step Box / Легкая походка',
 'Pulse Box / Пульс бокс',
 'Vision Box / Острое зрение',
 'Универсальный иммунобустер',
 'L-карнитин',
 'Глюкозамин и хондроитин',
 'Премиум набор для комплексного очищения организма',
 'IQ Box / Интеллект',
 'Хронобиологическая защита печени',
 'Набор Тригельм',
 'Энергомодулирующий комплекс в формате спрея',
 'Хронобиологическая защита сердца',
 'Хронобиологическая защита клеток мозга',
 'Хронобиологическая защита суставов',
 'Хронобиологическая защита зрения',
 'Витаминно-минеральный комплекс',
 'Новомин',
 '«Живокост», восстанавливающий бальзам форте',
 '«Уян Номо», бальзам для тела',
 'Спортивный восстанавливающий гель',
 'Бальзам с экстрактом окопника, глюкозамином и хондроитином',
 'Набор для комплексного очищения организма',
 'ЭПАМ 96 (урологический)',
 'ЭПАМ 900 (респираторный)',
 'ЭПАМ 7 (антивирусный)',
 'ЭПАМ 44 (кардиологический)',
 'ЭПАМ 4 (печеночный)',
 'ЭПАМ 31 (противофиброзный)',
 'ЭПАМ 24 (гинекологический)',
 'ЭПАМ 11 (желудочно-кишечный)',
 'ЭПАМ 1000 (неврологический)',
 'Природный инулиновый концентрат',
 'Лецитин из подсолнечника Vitamama Family (сироп)',
 "MEN'S BOX / Мужская сила",
 'Бьюти-комплекс «Трипептиды коллагена & Кофакторы»',
 'Ежовик гребенчатый и кордицепс для улучшения когнитивных функций',
 'Комплекс древесных грибов для поддержки иммунитета и антиоксидантной защиты',
 'Набор «Ясность и концентрация»',
 'Хронолонг',
 'Витамин D3',
 'Фиточай из диких трав № 6 (Защита печени)',
 'Фиточай из диких трав № 3 (Природный антистресс)',
 'Immunotops, жевательные таблетки со вкусом клюквы и витаминами A, C и D',
 'Мультихелат магния',
 'Natural Vitamins',
 'Mama Box / Здоровая мама',
 'Хронобиологическая защита зрения Синхровитал VII',
 'Хронобиологическая защита суставов Синхровитал VI',
 'Хронобиологическая защита печени Синхровитал IV',
 'НовоМин',
 'Набор 4 Wellness',
 'Органический йод',
 '3D Bone Vegan Cube',
 '3D Hair & Nails Cube',
 'Stress Re.Live',
 'Neurovision',
 'Natural Vitamins',
 '3D Protection Cube',
 'Мегавитамины',
 'Beauty Box / Красота и сияние',
 'Быстрорастворимый комплекс веганских BCAA (апельсин-манго)',
 'Keforma Vegan. Протеиновый веган-коктейль «Шоколад и кокос», 480 г',
 'Keforma Vegan. Протеиновый веган-коктейль «Капучино», 480 г',
 'Протеиновый коктейль WHEY PLUS 80 «Капучино», 480 г',
 'Keforma. Набор пробников протеиновых коктейлей',
 'Чистый рыбный коллаген (клубника-лайм) Fitness Catalyst',
 'Протеиновый коктейль WHEY PLUS 80 «Темный шоколад», 480 г',
 'Протеиновый коктейль WHEY PLUS 80 «Печенье с шоколадом», 480 г',
 'Протеиновый коктейль WHEY PLUS 80 «Клубника», 480 г',
 'Пептиды морского коллагена Fitness Catalyst (ваниль-черника)',
 'Таурин',
 'Пептиды морского коллагена Fitness Catalyst (оригинал)',
 'Напиток с аминокислотами (гуарана)',
 'Хромлипаза',
 'Комплекс аминокислот BCAA',
 'Чистый рыбный коллаген (манго) Fitness Catalyst',
 'Шейкер Keforma, объем 400 мл',
 'Корень, бальзам-концентрат для массажа',
 'Сывороточный протеин Fitness Catalyst (шоколадное печенье)',
 ' Сывороточный протеин Fitness Catalyst (ванильное мороженое)',
 'Сывороточный протеин Fitness Catalyst (тирамису)',
 'Сывороточный протеин Fitness Catalyst (черничный чизкейк)',
 'Хрустящие шарики Immunotops «Шоколадное печенье» с витамином С',
 'Напиток Beauty-микс (лесные ягоды) с коллагеном и гиалуроновой кислотой для сияния кожи',
 'Energy, драже с мультивитаминами (черника)',
 'Напиток Beauty-микс (персик-опунция) с коллагеном и биотином для молодости кожи',
 'Veggie, питательный коктейль (кофе-шоколад)',
 'Antistress-батончик (карамель и фундук)',
 'Immuno-батончик (вишня-банан)',
 'Bones & Joints-батончик (кокос)',
 'Anti-Age микс, напиток «Клубника»',
 'Veggie, питательный коктейль (персик-апельсин)',
 'Veggie, питательный коктейль (апельсин-лайм)',
 'Immuno-мармелад (тропик)',
 'Beauty-мармелад с биотином (персик-манго)',
 'Натуральный жевательный мармелад (черная смородина)',
 'Натуральный жевательный мармелад (апельсин)',
 'Напиток Immuno Drink (Защита иммунитета) «Лимон-имбирь»',
 'Напиток Antiox Drink «Облепиха-корица»',
 'Батончик Banana Mama (вишня-банан)',
 'Calcium-батончик Кокос',
 'Жевательный мармелад с облепихой и корицей',
 'Detox-батончик со вкусом «Манго»',
 'Гель для душа с экстрактом лепестков розы',
 'Ампульный концентрат «Возрождение» с детокс-эффектом для любого типа кожи',
 'Шелковистое молочко для очищения кожи',
 'Крем для тела с экстрактом лепестков розы',
 'Крем для лица увлажняющий',
 'Крем для рук увлажняющий Siberian Herbs',
 'Скраб для тела с кедром с ягодами Siberian Herbs',
 'Крем для лица ночной с антиоксидантами',
 'Крем для лица дневной SPF 15 с антиоксидантами',
 'Пилинг для кожи головы',
 'Набор зубных паст «Для семьи»',
 'Интеллектуальный ночной крем с мелатонином',
 'Интеллектуальный крем для лица с растительными пептидами',
 'Обновляющий ночной крем',
 'Дневной крем с растительными пептидами',
 'Гель для деликатного ухода',
 'Ампульный концентрат «Увлажнение и сияние» для сухой и обезвоженной кожи',
 'Крем-комплекс для лица для зрелой кожи, 50 мл',
 'Ампульный концентрат «Лифтинг и упругость» для зрелой кожи',
 'Крем для бритья, 100 мл',
 'Шампунь для волос и тела мужской, 250 мл',
 'Омолаживающая пептидная сыворотка с комплексом пре- и пробиотиков',
 'Масло массажное расслабляющее Siberian Herbs',
 'Фитомыло для женской интимной гигиены',
 'Гоммаж для рук очищающий',
 'Дезодорант для тела женский',
 'Сыворотка-автозагар Sun-Kissed Skin с растительным бронзатором ДГА для лица',
 'Крем для кожи вокруг глаз',
 'Спрей для ног против запаха с шалфеем, подорожником и дубом Siberian Herbs',
 'Гель для умывания с омега-кислотами',
 'Крем для лица с омега-кислотами и бета-каротином',
 'Антиоксидантный тоник для лица',
 'Крем для рук обновляющий Siberian Herbs',
 'Зубная паста для осветления эмали «Черника & Древесный уголь»',
 'Крем для ног увлажняющий с прополисом Siberian Herbs',
 'Сыворотка для роста волос',
 'Део-гель для женской интимной гигиены',
 'Крем для рук питательный Siberian Herbs',
 'Дезодорант для тела мужской',
 'Маска для лица отбеливающая, 50 мл',
 'Крем для лица обновляющий',
 'Крем для век с омега-кислотами и бета-каротином',
 'Молочко-автозагар Sun-Kissed Skin с растительным бронзатором ДГА',
 'FLUIDES Pretty, парфюмерная вода, 20 мл',
 'Пилинг для лица энзимный',
 'Солнцезащитный крем для тела SPF 30',
 'Восстанавливающий дневной крем для лица SPF 15 / PA +++',
 'Крем для тела с конопляным маслом',
 'Бальзам-тоник после бритья',
 'GREEN-мыло для кухни Siberian Herbs',
 'Маска для волос восстанавливающая Siberian Herbs',
 'Крем для проблемной кожи',
 'Дезодорант для тела',
 'Флюид матирующий с пре- и пробиотиками для лица',
 'Восстанавливающая ночная крем-маска с пре- и пробиотиками',
 'Шампунь против перхоти с шалфеем и янтарной кислотой',
 'Детская зубная паста Dino Dent с ксилитолом «Персик»',
 'Детская зубная паста Dino Dent с ксилитолом «Карамель»',
 'Крем для тела с экстрактом подорожника Siberian Herbs',
 'Интеллектуальный крем для лица с растительными пептидами',
 'Интеллектуальный ночной крем с мелатонином',
 'Нежная очищающая эмульсия для лица',
 'Успокаивающая сыворотка против несовершенств кожи',
 'Сыворотка для кожи головы с гиалуроновой кислотой',
 'Флюид-уход для волос с церамидами и протеинами',
 'Тоник для лица обновляющий',
 'Кондиционер для волос (увлажнение и блеск)',
 'Крем для лица (лифтинг и увлажнение)',
 'Крем для тела (увлажнение и упругость)',
 'Маска для лица (увлажнение и ровный тон)',
 'Сыворотка для лица (лифтинг и тонус)',
 'Шампунь (увлажнение и объем)',
 'Гель для умывания (тонус и увлажнение)',
 'Гель для душа (интенсивное увлажнение)',
 'Гель для душа с конопляным маслом Siberian Herbs',
 'Гель интимный на водной основе (клубника)',
 'Гель интимный на водной основе (без запаха)',
 'Dino Skino, крем-баттер',
 'Зубная паста с пребиотиком Свежесть и защита для чувствительных зубов',
 'Сыворотка с витамином С для снятия стресса кожи',
 'Балансирующий тоник для лица',
 'Сыворотка с гиалуроновой кислотой',
 'Балансирующая мицеллярная вода',
 'Кондиционер для объема волос (янтарная кислота и фитокератин)',
 'Гель для душа с экстрактом подорожника Siberian Herbs',
 'Шампунь для роста волос с ниацинамидом и перцем',
 'Шампунь восстанавливающий (церамиды и протеины)',
 'Кондиционер для волос восстанавливающий (церамиды и протеины)',
 'Горчичный гель для растираний',
 'Кондиционер для волос увлажняющий с гиалуроновой кислотой',
 'Шампунь увлажняющий с гиалуроновой кислотой',
 'Полирующий эксфолиант для лица',
 'Пилинг для кожи головы',
 'Выравнивающая ночная крем-маска',
 'Тонер для сияния кожи с AHA- & PHA-кислотами',
 'Восстанавливающий тонер с церамидами',
 'Омолаживающий тоник-софтнер',
 'Солнцезащитный крем для лица SPF 50',
 'Восстанавливающая эссенция с ниацинамидом 10%',
 'Мультиактивный крем для шеи и декольте',
 'Увлажняющий крем для век',
 'Питательный крем-бальзам для лица',
 'Увлажняющий крем-гель для лица',
 'Шампунь для волос и тела мужской',
 'Очищающее масло для умывания',
 'Мицеллярная вода для снятия макияжа',
 'Криолифтинговая маска для лица',
 'Пенка для умывания увлажняющая',
 'Обновляющий и выравнивающий пилинг для лица',
 'Масло для губ (цвет: сладкий мед)',
 'Масло для губ (цвет: дикая малина)',
 'The Body Lab, укрепляющий и моделирующий крем для тела',
 'Крем для повышения упругости кожи вокруг глаз',
 'Обновляющий ночной крем',
 'Пробуждающая сыворотка для лица',
 'Дневной крем с растительными пептидами',
 'Мягкий гель для умывания с пробиотиками',
 'Восстанавливающий гель с пантенолом',
 'Зубная паста Интенсивное укрепление эмали',
 'Зубная паста Антибактериальная защита',
 'CoreNRG, природная профилактическая зубная паста',
 'Увлажняющая помада для губ (обнаженная роза)',
 'Увлажняющая помада для губ (розовый бутон)',
 'Увлажняющая помада для губ (пыльная роза)',
 'Тонизирующий крем для ног',
 'Восстанавливающий бальзам',
 'Суперстойкая тушь для ресниц (коричневая)',
 'Блеск для губ (сочная ягода)',
 'Стойкий карандаш для бровей (насыщенный коричневый)',
 'Стойкий карандаш для бровей (тауп)',
 'Увлажняющий тонирующий крем SPF 15 (средний)',
 'Антицеллюлитный концентрат',
 'Крем для век обновляющий с Q10',
 'Гидрогелевые патчи для кожи вокруг глаз',
 'Бальзам-флорасептик «Эльбэшэн»',
 'Бальзам широкого спектра действия «Корень»',
 'Мерцающий блеск для губ (искрящийся нюд)',
 'Суперстойкая тушь для ресниц (ультрачерная)',
 'Увлажняющий тонирующий крем SPF 15 (светлый)',
 'Бальзам широкого спектра действия',
 'Набор «Белоснежная улыбка»',
 'Интеллектуальная сыворотка для лица с растительными пептидами',
 'Скраб для тела разогревающий с имбирем',
 'Масло массажное согревающее с имбирем и жгучей ванилью',
 'Гель для душа тонизирующий с имбирем',
 'Гель для душа увлажняющий с шалфеем и хмелем',
 'Гель для душа смягчающий с розмарином и пантенолом',
 'Парфюмированное мыло ручной работы «Макадамия & Миндальное масло»',
 'Парфюмированное мыло ручной работы «Имбирь & Ваниль»',
 'Парфюмированный крем для рук «Мандарин & Хвоя»',
 'Парфюмированный крем для рук «Имбирь & Ваниль»',
 'Набор «Заботимся о гигиене» с Vitamama',
 'Маска для рук обновляющая',
 'Пенка для интимной гигиены',
 'Интеллектуальная сыворотка для лица с растительными пептидами',
 'Део-гель для женской интимной гигиены',
 'Ампульный концентрат-миорелаксант',
 'Набор «Прополис»',
 'Шампунь для объема волос (янтарная кислота и фитокератин)',
 'Маска для волос восстанавливающая Siberian Herbs',
 'Очищающая глиняная маска с угольными гранулами',
 'Крем для рук смягчающий',
 'Масло массажное согревающее Siberian Herbs',
 'Дезодорант для тела мужской',
 'Дезодорант для тела женский',
 'Скраб для тела с кедром с ягодами Siberian Herbs',
 'Крем для век с омега-кислотами и бета-каротином',
 'Увлажняющий тонирующий крем SPF 15 (слоновая кость)',
 'Гель для деликатного ухода',
 'Сыворотка для роста волос',
 'Крем для лица с омега-кислотами и бета-каротином',
 'Солнцезащитный крем для тела SPF 30',
 'Гель для умывания с омега-кислотами',
 'Масло массажное расслабляющее Siberian Herbs',
 'Масло массажное тонизирующее Siberian Herbs',
 'Крем для рук увлажняющий Siberian Herbs',
 'Крем для рук питательный Siberian Herbs',
 'Крем для рук обновляющий Siberian Herbs',
 'Маска для лица матирующая',
 'Маска для лица очищающая',
 'Крем для бритья',
 'Бальзам-тоник после бритья',
 'Крем для проблемной кожи',
 'Бальзам для губ с конопляным маслом',
 'Увлажняющий гель для умывания',
 'Ампульный концентрат с бакучиолом',
 'Ампульный концентрат «Увлажнение»',
 'Ампульный концентрат «Возрождение»',
 'Пилинг для лица энзимный',
 'Маска для лица отбеливающая',
 'Крем для лица ночной с антиоксидантами',
 'Крем для лица дневной SPF 15 с антиоксидантами',
 'Антиоксидантный тоник для лица',
 'Фитомыло для женской интимной гигиены',
 '«Черника & древесный уголь», биоактивная зубная паста',
 'Зубная паста Натуральная защита',
 'Крем для лица обновляющий',
 'Дезодорант для тела',
 'Крем-комплекс для зрелой кожи',
 'Крем для лица увлажняющий',
 'Episode 26, парфюмерная вода, 1,5 мл',
 'Episode 26, парфюмерная вода',
 'Episode 23, парфюмерная вода, 1,5 мл',
 'Episode 23, парфюмерная вода',
 'Nuage Gardenia, парфюмерная вода, 1,5 мл',
 'Nuage Gardenia, парфюмерная вода',
 'Парфюмерная вода Arc-en-Ciel, 1.5 мл',
 'Arc-en-Ciel White, парфюмерная вода, 1,5 мл',
 'Arc-en-Ciel Pink, парфюмерная вода, 1.5 мл',
 'Arc-en-Ciel Red, парфюмерная вода, 1,5 мл',
 'Arc-en-Ciel Red, парфюмерная вода, 20 мл',
 'Arc-en-Ciel Pink, парфюмерная вода, 20 мл',
 'Arc-en-Ciel Emerald, парфюмерная вода, 20 мл',
 'Парфюмерная вода Arc-en-Ciel Ice Dance, 1,5 мл',
 'Arc-en-Ciel White, парфюмерная вода, 20 мл',
 'Подарочный набор изысканных ароматов «Секрет абсолютной женственности»',
 'Парфюмированный мист «Лесной орех & Бобы тонка» для ароматизации помещений и текстиля',
 'Arc-en-ciel Bloom, парфюмерная вода, 1,5 мл',
 'Парфюмерная вода Masala tea & Tobacco (Чай масала и Табак), 50 мл',
 'Парфюмерная вода Blue Matcha & Vetiver (Голубая матча и Ветивер), 50 мл',
 'Парфюмерная вода Neroli Tea & Bergamot (Чай нероли и Бергамот), 50 мл',
 'Парфюмерная вода Tea Blossom & Candy (Цветущий чай и леденцы), 50 мл',
 'Парфюмерная вода White Tea & Mimosa (Белый чай и Мимоза), 50 мл',
 'Lady Vogue Dream, парфюмерная вода, 20 мл',
 'Lady Vogue Soul, парфюмерная вода, 20 мл',
 'Парфюмерная вода White Tea & Mimosa (Белый чай и Мимоза), 1,5 мл',
 'Парфюмерная вода Tea Blossom & Candy (Цветущий чай и леденцы), 1,5 мл',
 'Парфюмерная вода Neroli Tea & Bergamot (Чай нероли и Бергамот), 1,5 мл',
 'Парфюмерная вода Blue Matcha & Vetiver (Голубая матча и Ветивер), 1,5 мл',
 'Парфюмерная вода Masala tea & Tobacco (Чай масала и Табак), 1,5 мл',
 'Dynamique (Динамика), духи-концентрат, 1.5 мл',
 '6 Sextum, духи-концентрат, 1,5 мл',
 'L’essence de Taiga, духи-концентрат, 1,5 мл',
 'Nuage Osmanthus, парфюмерная вода',
 'Nuage Osmanthus, парфюмерная вода, 1,5 мл',
 'Arc-en-ciel Bloom, парфюмерная вода, 20 мл',
 'L’essence d’Altai, духи-концентрат, 1,5 мл',
 'Nuage Freesia, парфюмерная вода',
 'Nuage Freesia, парфюмерная вода, 1,5 мл',
 'Духи-концентрат, Absolu (Абсолют), 1,5 мл',
 'Paradise Fruits & Jasmine, туалетная вода, 1,5 мл',
 'Paradise Fruits & Jasmine, туалетная вода',
 'White Flowers & Honey, туалетная вода, 1,5 мл',
 'White Flowers & Honey, туалетная вода',
 'Wild Raspberry & Bitter Orange, туалетная вода, 1,5 мл',
 'Wild Raspberry & Bitter Orange, туалетная вода',
 'Absolu (Абсолют), духи-концентрат, 20 мл',
 'Golden Amber & Midnight Saffron, парфюмерная вода, 1,5 мл',
 '9 Nonum, духи-концентрат, 1,5 мл',
 'FLUIDES Like God, парфюмерная вода, 20 мл',
 'FLUIDES So Good, парфюмерная вода, 20 мл',
 'Dark Vanilla & Cherry Blossom, парфюмерная вода, 1,5 мл',
 'Парфюмерная вода Arc-en-Ciel, 20 мл',
 'Dynamique (Динамика), духи-концентрат, 20 мл',
 'L’essence d’Altai, духи-концентрат, 20 мл',
 'FLUIDES I Do, парфюмерная вода, 20 мл',
 'FLUIDES Magic, парфюмерная вода, 20 мл',
 '1 Primum, духи-концентрат, 20 мл',
 'Red Rose & Oud, парфюмерная вода, 1,5 мл',
 'Neon Rose & Davana, парфюмерная вода, 1,5 мл',
 'Orange Rose & Vanilla, парфюмерная вода, 1,5 мл',
 'Pink Rose & Sea Salt, парфюмерная вода, 1,5 мл',
 'White Rose & Musk, парфюмерная вода, 1,5 мл',
 'Demi-Lune № 04, парфюмерная вода для мужчин, 1,5 мл',
 'Nuage, парфюмерная вода, 1,5 мл',
 'Demon du Ciel, парфюмерная вода, 1,5 мл',
 'Silver Wood & Oakmoss, парфюмерная вода, 1,5 мл',
 'Golden Violet & Amber Absolu, парфюмерная вода, 1,5 мл',
 'Парфюмерная вода Arc-en-Ciel Ice Dance, 20 мл',
 'L’essence de Taiga, духи-концентрат, 20 мл',
 'Golden Violet & Amber Absolu, парфюмерная вода',
 'Silver Wood & Oakmoss, парфюмерная вода',
 'Demon du Ciel, парфюмерная вода для женщин, 20 мл',
 'Episode 21, парфюмерная вода',
 'Nuage, парфюмерная вода',
 'Набор Aroma Box «Торжество цветочных мотивов»',
 'Demi-Lune № 04, парфюмерная вода для мужчин, 20 мл (лимитированный дизайн)',
 '9 Nonum, духи-концентрат, 20 мл',
 '6 Sextum, духи-концентрат, 20 мл',
 'Fresh Fig & Sparkling Tangerine, парфюмерная вода, 20 мл',
 'Golden Amber & Midnight Saffron, парфюмерная вода, 20 мл',
 'Dark Vanilla & Cherry Blossom, парфюмерная вода, 20 мл',
 'FLUIDES Touch Me, парфюмерная вода, 20 мл',
 'Neon Rose & Davana, парфюмерная вода, 50 мл',
 'Neon Rose & Davana, парфюмерная вода, 20 мл',
 'Red Rose & Oud, парфюмерная вода, 20 мл',
 'Red Rose & Oud, парфюмерная вода, 50 мл',
 'Orange Rose & Vanilla, парфюмерная вода, 20 мл',
 'Orange Rose & Vanilla, парфюмерная вода, 50 мл',
 'Pink Rose & Sea Salt, парфюмерная вода, 20 мл',
 'Pink Rose & Sea Salt, парфюмерная вода, 50 мл',
 'White Rose & Musk, парфюмерная вода, 20 мл',
 'White Rose & Musk, парфюмерная вода, 50 мл',
 'Demi-Lune № 04, парфюмерная вода для мужчин',
 'Episode 01, парфюмерная вода',
 'Episode 02, парфюмерная вода',
 'L’essence  d’Altai, духи-концентрат',
 '9 Nonum, духи-концентрат',
 '6 Sextum, духи-концентрат',
 '1 Primum, духи-концентрат',
 'Absolu (Абсолют), духи-концентрат',
 'Fresh Fig & Sparkling Tangerine, парфюмерная вода',
 'Golden Amber & Midnight Saffron, парфюмерная вода',
 'Dark Vanilla & Cherry Blossom, парфюмерная вода',
 'L’essence de Taiga, духи-концентрат',
 'Paradise Fruits & Jasmine, туалетная вода, 20 мл',
 'White Flowers & Honey, туалетная вода, 20 мл',
 'Wild Raspberry & Bitter Orange, туалетная вода, 20 мл',
 'Absolute Ego, парфюмерная вода для мужчин, 20 мл',
 'Подарочный набор мужских ароматов «Секрет неповторимого стиля»',
 'Парфюмированный мист «Имбирь & Ваниль» для ароматизации текстиля и помещений',
 'Набор Aroma Box «Стиль, достойный победителя»',
 'Набор Aroma Box «Оставляй неповторимое впечатление!»',
 'Episode 19, парфюмерная вода',
 'Набор Aroma Box «Вечером все желания становятся сильнее!»',
 'Episode 19, парфюмерная вода',
 'Dynamique (Динамика), духи-концентрат']

            }

In [49]:
# Сохранение в JSON
with open('category_article.json', 'w', encoding='utf-8') as file:
    json.dump(category_article, file, ensure_ascii=False, indent=4)
# Сохранение в JSON
with open('logik_data.json', 'w', encoding='utf-8') as file:
    json.dump(logik_data, file, ensure_ascii=False, indent=4)

In [50]:
import json

# Загрузка данных из JSON
with open('category_article.json', 'r', encoding='utf-8') as file:
    category_article = json.load(file)
with open('logik_data.json', 'r', encoding='utf-8') as file:
    logik_data= json.load(file)


In [35]:
class Logicif:
  def __init__(self, level=4, data=None):
    self.data=data
    self.level=level
  def if_else(self, text, level=None, data=None):

      """
      Проверьте наличие "текста" на указанных уровнях "данных".

      Параметры:
      - текст (str): текст для поиска.
      - данные (список, необязательно): список, содержащий различные уровни данных.

      ВОЗВРАТ:
      - int: найденный уровень (1, 2 b и тд), или 0, если не найден.
      """
      level_return = 0
      # Используйте предоставленные данные, если таковые имеются; в противном случае используйте self.data
      if data is None:
          data = self.data
      if level is None:
          level= self.level
            # Check each level for the presence of 'text'
      for i in range(1,level):
        if text in data[i]:
            #print(text)
            level_return=i
      return level_return

In [14]:
"""База промптов для разных ситуаций в диалоге первичный, повышающий, понижающий"""

class Prompt:
  def __init__(self, data=None):
    self.data=data

  def prompt_base(self, level, info_data, user_message=None):
    prompt={'приветствие':
        f"""Ты менеджер компании 'Siberian Wellnes', твоя задача проконсультировать пользователя по продуктам компании.
        Представься как нейроконсультант компании 'Siberian Wellness', презентуй компанию, вот информация о компании {info_data}, вот запрос пользователя: {user_message}.
         будь вежливым и вариативным, не придумывай факты, придерживайся предоставленной информации,
         старайся уложиться в 300 токенов""",

        'запрос':
        f"""Ты менеджер компании 'Siberian Wellnes', твоя задача проконсультировать пользователя по продуктам компании.
        (не надо представляться ты уже представился и поздоровался)
        Cпроси у пользователя в какой категории он
        рассматривает приобретение продукта, вот список категорий {info_data}, вот запрос пользователя: {user_message}. будь вежливым и вариативным, не придумывай факты, придерживайся предоставленной информации,
         старайся уложиться в 300 токенов""",

        'подкат':
        f"""Ты менеджер компании 'Siberian Wellnes', твоя задача проконсультировать пользователя по продуктам компании.
        (не надо представляться ты уже представился и поздоровался)
        Пользователь определился с категорией: {user_message}, примени подобный оборот вариативно, [я вижу вас интересует эта категория {info_data}],
        предложи расмотреть подкатегории в этом направлении, представь пользователю эти категории, будь вежливым и вариативным, не придумывай факты, придерживайся предоставленной информации,
        старайся уложиться в 450 токенов""",
        'прекат':
        f"""Ты менеджер компании 'Siberian Wellnes', твоя задача проконсультировать пользователя по продуктам компании.
        (не надо представляться ты уже представился и поздоровался)
        Пользователь выбрал подкатегорию товаров {info_data}, вот его запрос{user_message}: примени подобный оборот вариативно опираясь на эти данные,
         [Из данной подкатегории {info_data} я мог бы вам порекомендовать],
         предложи рассмотреть этот список товаров, будь вежливым и вариативным, не придумывай факты, придерживайся предоставленной информации, старайся уложиться в 350 токенов""",
        'презент':
        f"""Ты менеджер компании 'Siberian Wellnes', твоя задача проконсультировать пользователя по продуктам компании.
        (не надо представляться ты уже представился и поздоровался)
        Пользователь определился с товаром {info_data} и желает получить консультацию, вот его вопрос {user_message}, изучи информацию о товаре,
         вариативно в формате [Я благодарен вам за ваш выбор нашего продукта{info_data}] и в ооответствии с запросом пользователя, предоставь информацию,
         не выдумывай факты, придерживайся предоставленной информации, старайся уложиться в 350 токенов, будь заинтересован в продаже,
         предложи в конце презентации помочь связаться с менеджером для приобретения товара""",
        'откат':
        f"""Ты менеджер компании 'Siberian Wellnes', твоя задача проконсультировать пользователя по продуктам компании.
        (не надо представляться ты уже представился и поздоровался)
        Пользователь решил что он хотел бы выбрать что нибудь еще, вот его ответ:{user_message}, вот история вашего общения предложи чтонибудь из предыдущих сообщений  {info_data},
         Будь вежлив в формате,
         вариативно [Я ценю ваш выбор, и могу предложить определиться с категорией ранее мы с вами рассматривали эти варианты]""",
        'финал':
        f"""Ты менеджер компании 'Siberian Wellnes', твоя задача проконсультировать пользователя по продуктам компании.
        (не надо представляться ты уже представился и поздоровался)
        Пользователь определился с выбором и готов  приобрести товар, сообщи вариативно [Я благодарен за ваш выбор{info_data},
         если вы желаете приобрести этот товар, оставьте ваши контактные данные и наш менеджер свяжеться с вами] """
         }
    return prompt[level]


In [15]:
# класс продукт, создан для размещения списка продуктов, для дальнейшего извлечения данных по запросу на данный момент не задействован, как задел на будущее.
class Product:
  def __init__(self, sku, name, shortName, description, shortDescription ,howToUse, contraindications, productComponent, productComponentRaw, options, price, isAction, categories, topCategory, thumbnails):
    self.sku=sku
    self.name=name
    self.shortName=shortName
    self.description=description
    self.shortDescription=shortDescription
    self.howToUse=howToUse
    self.contraindications=contraindications
    self.productComponent=productComponent
    self.productComponentRaw=productComponentRaw
    self.options=options
    self.price=price
    self.isAction=isAction
    self.categories=categories
    self.topCategory=topCategory
    self.thumbnails=thumbnails
  def get_info(self):
    return (self.sku, self.name, self.description, self.shortDescription, self.howToUse, self.contraindications, self.productComponent, self.productComponentRaw, self.options, self.price, self.isAction, self.categories, self.topCategory, self.thumbnails)
  def get_sku(self):
    return self.sku
  def get_name(self):
    return self.name
  def get_description(self):
    return self.description
  def get_shortDescription(self):
    return self.shortDescription
  def get_contraindications(self):
    return self.contraindications
  def get_productComponent(self):
    return self.productComponent
  def get_productComponentRaw(self):
    return self.productComponentRaw
  def get_options(self):
    return self.options
  def get_price(self):
    return self.price
  def get_isAction(self):
    return self.isAction
  def get_categories(self):
    return self.categories
  def get_topCategory(self):
    return self.topCategory
  def get_thumbnails(self):
    return self.thumbnails
#id_1234 = Product('Мыло', '1234', 'https://ru.ru/1234', 'мыльная основа на растительном сырье, ароматические масла, натуральный краситель бета каротин', 'мыло ручной работы, с ароматом лаванды, для ежедневного применения', '300', '5', 'гигиена')
#print(id_1234.get_structure())
#print(id_1234.get_info())

In [16]:
# класс с инструментами для выявления в тексте запроса сущьностей, стабилизирует текст запроса и передаваемых данных
class Response_user_text:
    def __init__(self, data_response=None, user_message=None, model='paraphrase-MiniLM-L6-v2'):
        self.data_response = data_response
        self.user_message = user_message
        self.model = model

    def key_load(self, data):
        keys = list(data.keys())
        keys = [key.lower() for key in keys]  # Приводим ключи к нижнему регистру
        return keys

    def clean_string(self, input_string):
        # Удаляем все символы, кроме букв, цифр и пробелов
        cleaned_string = re.sub(r'[^A-Za-zА-Яа-я0-9 ]', '', input_string)
        return cleaned_string.lower()

    def response_user_message(self, user_message, data=None):
        # Загрузка модели по умолчанию 'paraphrase-MiniLM-L6-v2'
        model = SentenceTransformer(self.model)
        # Очистка сообщения пользователя
        user_message = self.clean_string(user_message)
        # Получаем ключи
        if data is None:
            data = self.data_response
        keys = data
        # Проверка на пустые данные
        if not keys:
            return None, user_message
        # Загружаем в модель сообщение и список слов
        key_embeddings = model.encode(keys)
        message_embedding = model.encode(user_message)
        # Вычисление сходства
        cosine_scores = util.pytorch_cos_sim(message_embedding, key_embeddings)
        # Возвращаем самый высокий балл
        best_match_index = cosine_scores.argmax()
        # Возвращаем самый достоверный ключ и сообщение пользователя
        return keys[best_match_index]#, user_message

In [17]:
# класс для извлечения данных с сайта, выбирает с сайта методом прохода по всем категориям все данные
# сохраняет данные в переменной, выгружает на локальное хранилище, что бы не обращатся к сайту с запросами каждый раз
class Extruder:
    def __init__(self, headers, categories, size, url_site):
        self.headers = headers
        self.categories = categories
        self.size = size
        self.url_site = url_site
    def one_price(self, articul, url=None): # извлекает один товар ориентирован на class Product
        if url==None:
          url=f'{self.url_site}{articul}'
          response = requests.get(url, headers=self.headers)
        if response.status_code == 200:
           # Создаем объект BeautifulSoup для парсинга HTML
           soup = BeautifulSoup(response.text, 'html.parser')
           # Извлечение значения атрибута value
           param_tag = soup.find('param', {'get-dom-data': 'product.data'})
           value = param_tag['value']
           # Преобразование строки JSON в объект Python
        json_data = json.loads(value)
        prod={articul:{'sku': json_data[articul]['sku'],
              'name':json_data[articul]['name'],
              'shortName': json_data[articul]['shortName'],
              'description': json_data[articul]['shortName'],
              'shortDescription': json_data[articul]['shortDescription'],
              'howToUse': json_data[articul]['howToUse'],
              'contraindications': json_data[articul]['contraindications'],
              'productComponent': json_data[articul]['productComponent'],
              'productComponentRaw': json_data[articul]['productComponentRaw'],
              'options': json_data[articul]['options']['attributes'],
              'price': json_data[articul]['options']['price'],
              'isAction': json_data[articul]['options']['isAction'],
              'categories': json_data[articul]['categories'],
              'topCategory': json_data[articul]['topCategory'],
              'thumbnails': json_data[articul]['thumbnails'][0][self.size]}}
        return prod

    def one_category(self, url): #извлекает одну категорию
        """Извлекает json с данными по одной категории."""
        response = requests.get(url, headers=self.headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            param_tag = soup.find('param', {'catalog-brand': 'catalog.brand'})
            value = param_tag['value']
            json_data = json.loads(value)

            product_names = [product['name'] for product in json_data]
            photo_url = [product['images'][0][self.size] for product in json_data]
            id_names = [product['code'] for product in json_data]

            photo_dict = {id_name: photo for id_name, photo in zip(id_names, photo_url)}
            product_dict = {id_name: product_name for id_name, product_name in zip(id_names, product_names)}
            return product_dict, photo_dict
        return {}, {}

    def all_categories(self):
        """Извлекает и соединяет данные из разных категорий."""
        cat_prod = {}
        phot_prod = {}

        for name in self.categories:
            products, photos = self.one_category(self.categories[name])
            cat_prod.update(products)
            phot_prod.update(photos)

        return cat_prod, phot_prod
    def all_products(self):
        all_base_sku=[]
        names, photos = self.all_categories()
        key_names=list(names.keys())
        url=self.url_site
        for name in key_names:
            one_price = self.one_price(f'{url}{name}')
            all_base_sku.append(one_price)
        return all_base_sku
    def load_all_products(self): # загружает ранее соззданный список товаров если он не создан то запрашивает создать его
        # loading
        if os.path.isfile('all_prod.pickle'):
            with open('all_prod.pickle', 'rb') as handle:
                all_prod = pickle.load(handle)
        else:
            all_prod =self.all_products()
            # saving
            with open('all_prod.pickle', 'wb') as handle:
                pickle.dump(all_prod, handle, protocol=pickle.HIGHEST_PROTOCOL)
        return all_prod

In [18]:
class Keymaster:
  def __init__(self, data):
    self.data=data
  def find_values(self, target_key, data=None):
      if data is None:
        data=self.data
      """
      Рекурсивно ищет значения по указанному ключу target_key в многоуровневом словаре.

      data: dict - исходный словарь
      target_key: str - ключ, который нужно найти
      """
      results = []  # Список для хранения найденных значений

      # Проверяем, является ли data словарем
      if not isinstance(data, dict):
          raise ValueError("data должно быть словарем")

      for key, value in data.items():
          if key == target_key:
              results.append(value)  # Добавляем значение в список
          elif isinstance(value, dict):  # Если значение — вложенный словарь
              results.extend(self.find_values(target_key, value))  # Рекурсивный вызов

      return results
  def find_values_depth(self, target_key, data=None, depth=0):
      if data is None:
          data = self.data

      """
      Рекурсивно ищет значения по указанному ключу target_key в многоуровневом словаре.

      data: dict - исходный словарь
      target_key: str - ключ, который нужно найти
      depth: int - текущая глубина вложенности
      """
      results = []  # Список для хранения найденных значений
      max_depth = 0  # Максимальная глубина, на которой найден ключ

      # Проверяем, является ли data словарем
      if not isinstance(data, dict):
          raise ValueError("data должно быть словарем")

      for key, value in data.items():
          if key == target_key:
              # Если мы нашли ключ на текущем уровне
              if depth > max_depth:
                  results = [value]  # Сбрасываем результаты и добавляем новое значение
                  max_depth = depth  # Обновляем максимальную глубину
              elif depth == max_depth:
                  results.append(value)  # Добавляем значение на том же уровне

          elif isinstance(value, dict):  # Если значение — вложенный словарь
              sub_results = self.find_values_depth(target_key, value, depth + 1)  # Рекурсивный вызов
              if sub_results:
                  # Если в подрезультатах есть значения, обновляем результаты и максимальную глубину
                  results.extend(sub_results)

      return results
  def keys_level(self, dict_level, data=None):
      """Рекурсивно извлекает все ключи по заданному уровню словаря"""
      if data is None:
          data = self.data
      if dict_level == 0:
          return list(data.keys())  # Возвращаем ключи на текущем уровне
      keys = []  # Список для хранения ключей на нужном уровне
      for key, value in data.items():
          if isinstance(value, dict):  # Проверяем, является ли значение словарем
              keys.extend(self.keys_level(dict_level - 1, value))  # Рекурсивный вызов для вложенного словаря

      return keys

  def all_values(self, data=None):
      """рекурсивно вынимает все значения словаря"""
      if data is None:
          data = self.data
      values = []
      for value in data.values():
          if isinstance(value, dict):
              values.extend(self.all_values(value))  # Рекурсивный вызов для вложенного словаря
          else:
              for val in value:
                  #values.append(value)
                  values.append(val)
      return values
  def all_keys(self, data=None):
      """рекурсивно вынимает все ключи словаря"""
      if data is None:
          data = self.data
      keys = []
      for key, value in data.items():
          keys.append(key)
          if isinstance(value, dict):
              keys.extend(self.all_keys(value))  # Рекурсивный вызов для вложенного словаря
      return keys
  def find_key_level(self, target_key, data=None, current_level=0):
      if data is None:
          data = self.data
      """
      Определяет уровень заданного ключа в многоуровневом словаре.

      target_key: str - ключ, уровень которого нужно определить
      data: dict - исходный словарь
      current_level: int - текущий уровень вложенности (по умолчанию 0)

      Возвращает уровень ключа или None, если ключ не найден.
      """
      if not isinstance(data, dict):
          return None  # Если data не является словарем, возвращаем None

      if target_key in data:
          return current_level  # Ключ найден на текущем уровне

      for key, value in data.items():
          if isinstance(value, dict):  # Если значение — вложенный словарь
              level = self.find_key_level(target_key, value, current_level + 1)  # Рекурсивный вызов
              if level is not None:
                  return level  # Если ключ найден на более глубоком уровне, возвращаем его уровень

      return None  # Если ключ не найден в словаре

  def find_deepest_values(self, target_key, data=None):
      if data is None:
          data = self.data  # Используйте данные, переданные в конструктор

      """
      Рекурсивно ищет значения по указанному ключу target_key в многоуровневом словаре.
      Если ключ повторяется на нескольких уровнях, углубляется до самого глубокого уровня
      и возвращает конечные значения.

      data: dict - исходный словарь
      target_key: str - ключ, который нужно найти
      """
      # Проверяем, является ли data словарем
      if not isinstance(data, dict):
          raise ValueError("data должно быть словарем")

      for key, value in data.items():
          if key == target_key:
              # Если нашли ключ и значение — это словарь, продолжаем поиск внутри
              if isinstance(value, dict) and target_key in value:
                  return self.find_deepest_values(target_key, value)
              # Если значение — это список или другой конечный объект, возвращаем его
              elif isinstance(value, list):
                  return value
              else:
                  return value

          # Если значение — вложенный словарь, продолжаем поиск рекурсивно
          if isinstance(value, dict):
              result = self.find_deepest_values(target_key, value)
              if result is not None:
                  return result

      return None  # Если ничего не найдено
  def find_deepest_key_level(self, key_to_count, data=None):
      count = 0
      if data is None:
          data = self.data  # Используйте данные, переданные в конструктор
      # Проверяем, есть ли искомый ключ в текущем словаре
      if key_to_count in data:
        count += 1

      # Рекурсивно обходим все вложенные словари
      for key, value in data.items():
            if isinstance(value, dict):
              count += self.find_deepest_key_level(key_to_count, value)

      return count

In [19]:
class BotopenAI:
    def __init__(self, client, model, messages, max_tokens, temperature):
        self.client=client
        self.chat_completion= client.chat.completions.create(
            model=model,
            messages=messages,
            max_tokens=max_tokens,
            temperature=temperature)
    def used_bot(self):
      chat=self.chat_completion
      responses = chat
      response = responses.choices[0].message.content
      return response

In [20]:
extruder=Extruder(headers,category_urls,'medium',url_site)

In [21]:
prompt=Prompt()

In [22]:
keys=Keymaster(data=category_article)

In [23]:
response=Response_user_text(data_response= keys.all_values(data=logik_data), model='paraphrase-multilingual-MiniLM-L12-v2') #показывает наилучший результат
#response=Response_user_text(model='paraphrase-multilingual-mpnet-base-v2')
#response=Response_user_text()
#response=Response_user_text('xlm-roberta-large')
#response=Response_user_text(model='intfloat/multilingual-e5-large')

In [24]:
if_text=Logicif(9,data=logiс_data)

In [28]:
fio=['фамилия','имя','отчество']

In [51]:
extruder.load_all_products()[0]['500940']

{'sku': '500940',
 'name': ' Immuno, медовая паста с душицей и чабрецом для укрепления иммунитета',
 'shortName': 'Yoo Gо',
 'description': 'Yoo Gо',
 'shortDescription': 'Концентрат пользы и вкуса для твоего иммунитета – в ярком медовом миксе с соками яблока, вишни и черной моркови! Чабрец, боярышник и душица усиливают эффект, повышая защитные силы организма и помогая противостоять вирусам. ',
 'howToUse': '<p>Принимать по 1-2 чайной ложки 1-2 раза в день.</p>',
 'contraindications': '',
 'productComponent': '<p>Мед, сок яблочный концентрированный, сок вишневый концентрированный, плоды шиповника, экстракт плодов боярышника, экстракт чабреца, экстракт душицы, сок черной моркови концентрированный.</p>\n<p>Возможно наличие следов горчицы, кунжута, сельдерея, молока и продуктов их переработки.</p>\n<p><strong>Средние значения пищевой и энергетической ценности на 100 г:</strong> <br />белки &ndash; 0,1 г, <br />жиры &ndash; 0,03 г, <br />углеводы &ndash; 70,9 г; <br />энергетическая ценнос

In [56]:
s=if_text.if_else('протеины')
print(type(s))

<class 'int'>


In [52]:
keys.find_values_depth('протеины')

[['501457', '501456', '501455', '108221', '501388', '501389', '108310']]

In [29]:
user_message = "Константин"

user_response_message = response.response_user_message(user_message=user_message, data=fio)
if_text.if_else(user_response_message)
user_response_message

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

'отчество'

In [ ]:
"""Возможны реакции бота и данные которые должны быть переданы в этом случае
(6) приветствие
В данном случае желательно проверить, наличие истории в диалоге и произвести приветствие если диалог первичный и в истории пусто
пользователь заинтересован в информации о компании, бот  выдает информацию о компании
history
info_data = company данные по компании
(7)Запрос
В данном случае желательно проверить, наличие истории в диалоге и произвести приветствие если диалог первичный и в истории пусто
так же ознакомить пользователя с информацией о категориях товара
history
nfo_data = keys.keys_level(0) это список основных категорий
(1)подкат
В данном случае желательно проверить, наличие истории в диалоге и произвести приветствие если диалог первичный и в истории пусто
Выдача подкатегорий в запрашиваемой категории товаров
history
lev=keys.find_deepest_key_level(user_response_message)
dat=keys.find_values(user_response_message)
res=keys.keys_level(lev,dat[0])
прекат
В данном случае желательно проверить, наличие истории в диалоге и произвести приветствие если диалог первичный и в истории пусто
выдать пользователю список товаров в  данной подкатегории
презент
В данном случае желательно проверить, наличие истории в диалоге и произвести приветствие если диалог первичный и в истории пусто
Презентация товара, вовлечение в покупку
откат
Передать сети данные истории диалога, по хорошему вывести диалог на шаг назад в прекат.
 """

In [31]:
prod, key_prod = extruder.load_all_products()

In [32]:
def art_name(name, prod, key_prod):
  for key in key_prod:
   if prod[key]['name'] == name:
    return key


In [37]:
#user='позвоните менеджера пожалуйста'
#s= response.response_user_mesage(user_message=user_message, data=keys.all_values(data=logik_data))
def return_date_request(request):
  global history
  global max_tokens
  x=category_article
  resp= response.response_user_message(user_message=request)
  key_ = keys.find_deepest_key_level(resp)
  link=None
  if_ = if_text.if_else(resp)
  if keys.find_deepest_key_level(resp) == 1 and if_text.if_else(resp) == 1:
    x = keys.find_values_depth(resp)
    re = keys.keys_level(0,x[0])
    if_='подкат'
  if keys.find_deepest_key_level(resp) == 2 and if_text.if_else(resp) == 2:
    x = keys.find_values_depth(resp)
    x = keys.find_values_depth(resp, x[0])
    re = keys.keys_level(0,x[0])
    if_='подкат'
    max_tokens = 550
  if keys.find_deepest_key_level(resp) == 3 and if_text.if_else(resp) == 3:
    x = keys.find_deepest_values(resp)
    name=[]
    all_prod=extruder.load_all_products()
    for key in x:
      name.append(all_prod[0][key]['name'])
    re = name
    if_='прекат'
  if keys.find_deepest_key_level(resp) != 3 and if_text.if_else(resp) == 3:
    x = keys.find_deepest_values(resp)
    name=[]
    all_prod=extruder.load_all_products()
    for key in x:
      name.append(all_prod[0][key]['name'])
    re = name
    if_='прекат'
  if keys.find_deepest_key_level(resp) != 2 and if_text.if_else(resp) == 2:
    x = keys.find_values_depth(resp)
    x = keys.all_keys(data=x[0])
    re = x
    if_ = 'подкат'
    max_tokens = 550
  if keys.find_deepest_key_level(resp) != 1 and if_text.if_else(resp) == 7:
    re = keys.keys_level(0,category_article)
    if_='запрос'
  if keys.find_deepest_key_level(resp) != 1 and if_text.if_else(resp) == 6:
    re = company
    if_='приветствие'
  if keys.find_deepest_key_level(resp) != 1 and if_text.if_else(resp) == 4:
    re = history
    if_='откат'
  if keys.find_deepest_key_level(resp) != 1 and if_text.if_else(resp) == 5:
    re = 'manager call'
    if_='финал'
  if keys.find_deepest_key_level(resp) != 1 and if_text.if_else(resp) == 8:
    re = extruder.one_price(art_name(resp, prod, key_prod))
    art_= art_name(resp, prod, key_prod)
    link = re[art_]['thumbnails']
    if_='презент'
  print(key_)
  print(if_)
  bot=BotopenAI(client=client,model=model, messages=[{"role": "user", "content": prompt.prompt_base(level=if_, info_data = re, user_message = request)}],
                  max_tokens=max_tokens, temperature=temperature)
  text = bot.used_bot()
  history.append((text, user_message))
  return text, link , if_

In [48]:
# тут блок телеграмм
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Отправляет приветственное сообщение при запуске бота."""
    welcome_message = (
        "Добро пожаловать! Я бот компании 'Сибирское здоровье'.\n"
        "Все управление происходит через запросы. Просто отправьте свой запрос."
    )
    await update.message.reply_text(welcome_message)

async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Обрабатывает текстовые сообщения от пользователя."""
    user_message = update.message.text  # Получаем текст сообщения
    response, link, if_ = return_date_request(user_message)  # Получаем ответ от функции return_date_request

    if if_ == 'презент':
        # Если условие 'презент', отправляем изображение
        photo_url = link  #URL изображения
        await update.message.reply_photo(photo=photo_url)

        # Отправляем текст отдельным сообщением
        text = response
        await update.message.reply_text(text)
    else:
        # В остальных случаях отправляем текстовый ответ
        await update.message.reply_text(response)
def main() -> None:
    """Запускает бота."""
    application = Application.builder().token('ваш ключ').build()

    # Обработчики команд и сообщений
    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))  # Обработка текстовых сообщений
    #application.add_handler(MessageHandler(filters.PHOTO, handle_photo))  # Обработка фото

    # Запуск бота
    application.run_polling()

if __name__ == '__main__':
    main()

RuntimeError: Cannot close a running event loop

In [38]:
user='Dino Vitamino, сироп с витаминами и минералами'
re_, link , if_=return_date_request(user)

0
презент


In [39]:
re_

'Я благодарен вам за выбор нашего продукта - **Dino Vitamino, сироп с витаминами и минералами** от **Vitamama**. Этот фруктовый микс заряжен настоящей дино-энергией и содержит главные витамины для активного роста детей в 100% дозировке. В состав входит половина дневной нормы железа, цинка и витамина D₃, что делает его идеальным для дошкольников. Формула разработана совместно с экспертами DSM Nutritional (Швейцария) и не содержит добавленного сахара и консервантов.\n\n### Как использовать:\nРекомендуется давать детям от 3 лет по одной чайной ложке 1-2 раза в день на протяжении месяца. Лайфхак: добавьте сироп в чай, морс или кашу вместо варенья!\n\n### Активные компоненты:\nСироп включает в себя фруктоолигосахариды, яблочный и апельсиновый соки, а также витаминно-минеральный премикс с витаминами A, C, D и многими другими. \n\n### Пищевая ценность на 100 г:\n- Белки: 0 г\n- Жиры: 0 г\n- Углеводы: 71,3 г\n- Энергетическая ценность: 1469 кДж /'

In [ ]:
#history=[]
#def generate_bot(user_message):
#  if if_text.if_else(user_response_message) == 1
#    bot=BotopenAI(client=client,model=model, messages=[{"role": "user", "content": prompt.prompt_base(level='подкат', info_data = res, user_message = user_message)}],
#                  max_tokens=max_tokens, temperature=temperature)
#    text = bot.used_bot()
#    history.append((text, user_message))
#  elif if_text.if_else(user_response_message)==3:
#    name=[]
#    all_prod=extruder.load_all_products()
#    for key in keys.find_deepest_values(user_response_message):
#      name.append(all_prod[0][key]['name'])
#    bot=BotopenAI(client=client, model=model, messages=[{"role": "user", "content": prompt.prompt_base(level='прекат', info_data = name, user_message = user_message)}],
#                  max_tokens=450, temperature=temperature)
#    text = bot.used_bot()
#    history.append((text, user_message))
#  elif if_text.if_else(user_response_message)==4:
#    bot=BotopenAI(client=client, model=model, messages=[{"role": "user", "content": prompt.prompt_base(level='откат', info_data = history, user_message = user_message)}],
#                  max_tokens=max_tokens, temperature=temperature)
#    text = bot.used_bot()
#    history.append((text, user_message))
#  elif if_text.if_else(user_response_message)==6:
#    bot=BotopenAI(client=client, model=model, messages=[{"role": "user", "content": prompt.prompt_base(level='приветствие', info_data = company, user_message = user_message)}],
#                  max_tokens=max_tokens, temperature=temperature)
#    text = bot.used_bot()
#    history.append((text, user_message))
#  elif if_text.if_else(user_response_message)==7:
#    bot=BotopenAI(client=client, model=model, messages=[{"role": "user", "content": prompt.prompt_base(level='запрос', info_data = keys.keys_level(0), user_message = user_message)}],
#                  max_tokens=max_tokens, temperature=temperature)
#    text = bot.used_bot()
#    history.append((text, user_message))
#  elif if_text.if_else(user_response_message)==8:
#    bot=BotopenAI(client=client, model=model, messages=[{"role": "user", "content": prompt.prompt_base(level='презент', info_data = extruder.one_price(art_name(user_response_message, prod, key_prod)), user_message = user_message)}],
#                  max_tokens=max_tokens, temperature=temperature)
#    text = bot.used_bot()
#    history.append((text, user_message))
#  return text

In [ ]:
link

'https://static.siberianhealth.com/public/projects/shopen/images/50/_resize/500927_dbf183f7_fit_300_300.jpg'

In [ ]:
res = extruder.one_price(art_name('Dino Vitamino, сироп с витаминами и минералами', prod, key_prod))
key_= art_name('Dino Vitamino, сироп с витаминами и минералами', prod, key_prod)
res[key_]['thumbnails']

'https://static.siberianhealth.com/public/projects/shopen/images/50/_resize/500927_dbf183f7_fit_300_300.jpg'

In [ ]:
key_

'500927'

In [ ]:
user='что у вас есть для поддержки органов'
re_, key_, if_=return_date_request(user)

1
2


In [ ]:
re_

['защита печени',
 'защита почек',
 'защита сердца',
 'защита сосудов',
 'поддержка жкт',
 'микрофлора кишечника']

In [ ]:
user='защита печени'
re_, key_, if_=return_date_request(user)

1
3


In [ ]:
#def count_key_occurrences(d, key_to_count):
#    count = 0
#
#    # Проверяем, есть ли искомый ключ в текущем словаре
#    if key_to_count in d:
#        count += 1
#
#    # Рекурсивно обходим все вложенные словари
#    for key, value in d.items():
#        if isinstance(value, dict):
#            count += count_key_occurrences(value, key_to_count)
#
#    return count
#
## Пример использования:
#key_to_count = 'для спорта'
#occurrences = count_key_occurrences(category_article, key_to_count)
#print(f"Ключ '{key_to_count}' встречается {occurrences} раз(а).")

Ключ 'для спорта' встречается 2 раз(а).


In [ ]:
#def job_dikt(data, user_response):
#    global user_response_data
#    user_response_data.append(user_response)
#    # Доступ к вложенному элементу с использованием списка ключей
#    result = data
#    for key in user_response_data:
#        result = result[key]            # Пошаговый доступ по ключам
#    if isinstance(result, dict):        # Проверяем, является ли результат словарем
#        result = list(result.keys())   # Выводим ключи словаря
#    elif isinstance(result, list):      # Проверяем, является ли результат списком
#        result                          # Выводим сам список
#    else:
#        result = "Результат не является ни списком, ни словарем."
#    return result

In [ ]:
re_

['Лецитин подсолнечника с МСТ-маслом и бета-каротином',
 'S-ацетил-L-глутатион',
 'Detox Pro.Active',
 'Detox Pro.Active',
 'Премиум набор для комплексного очищения организма',
 'Хронобиологическая защита печени',
 'ЭПАМ 4 (печеночный)',
 'Хронобиологическая защита печени Синхровитал IV']

In [ ]:
user='S-ацетил-L-глутатион'
re_, key_, if_=return_date_request(user)

0
8


In [ ]:
re_

{'411382': {'sku': '411382',
  'name': 'Антицеллюлитный концентрат',
  'shortName': 'Experalta Platinum ',
  'description': 'Experalta Platinum ',
  'shortDescription': 'Инновация: антицеллюлитный комплекс BrownSlim запускает процесс сжигания жиров, повышая эффективность тренировок. Результат: фигура более подтянутая, кожа – ровная и гладкая.',
  'howToUse': '<p><span class="">Наносить на проблемные участки тела легкими массажными движениями. Оказывает сильное разогревающее действие. Рекомендуется использовать 2 раза в день курсом не менее 20 дней подряд, затем продолжить использование 1 раз в день для поддержания эффекта. </span></p>',
  'contraindications': '',
  'productComponent': '<p>Ингредиенты: вода, глицерин, мочевина, диметилизосорбид, бензиловый спирт, карбомер, кофеин, витамин РР, аллантоин, эфир ванили, эфирное масло цедры апельсина, экстракт плюща, лецитин, глицериллаурат, эфирное масло мяты, экстракт гарцинии камбоджийской, экстракт конского каштана, гидроксид натрия, бен

In [ ]:
#это категория запрос
user='какие категории товаров у вас представленны'
#s= response.response_user_mesage(user_message=user_message, data=keys.all_values(data=logik_data))
s= response.response_user_message(user_message=user)
print(s)
print(if_text.if_else(s))
keys.keys_level(0,x)

какие товары у вас есть
7


['витамины и добавки',
 'для спорта',
 'здоровое питание',
 'для поддержания красоты',
 'парфюмерия',
 'чаи и напитки',
 'бальзамы',
 'сопутствующие товары']

In [ ]:
user_message = "Добрый день, расскажите про вашу компанию"
print(generate_bot(user_message))

Я благодарен вам за ваш выбор нашего продукта — Дневного крема с растительными пептидами из линии Experalta Platinum. Этот крем создан для защиты кожи от агрессивного воздействия городской среды, поддержания оптимального уровня увлажнения и питания. В его состав входят растительные пептиды из кедровых орехов и сибирского женьшеня, а также натуральный комплекс HerbaGlow® NRG.

Крем легко наносится легкими похлопывающими движениями на чистую кожу лица, шеи и декольте. Для достижения наилучшего эффекта рекомендуется использовать его вместе с ампульными концентратами Experalta Platinum. Обратите внимание, что продукт предназначен только для наружного применения.

Состав крема включает множество активных ингредиентов, таких как ниацинамид, экстракты редких растений и коллоидная платина, которые способствуют синтезу коллагена и повышению эластичности кожи. Уникальная формула помогает уменьшить морщины и выровнять тон кожи.

Если у вас возникнут дополнительные вопросы или вы хотите приобрести

In [ ]:
user_message = "что бы вы могли мне расказать о ваших товарах"
print(generate_bot(user_message))

Здравствуйте! Рад вас видеть. У нас широкий ассортимент продуктов, и я с удовольствием помогу вам с выбором. Чтобы дать вам наиболее релевантную информацию, подскажите, в какой категории вы рассматриваете приобретение продукта? У нас есть такие категории, как:

- Витамины и добавки
- Для спорта
- Здоровое питание
- Для поддержания красоты
- Парфюмерия
- Чаи и напитки
- Бальзамы
- Сопутствующие товары

Жду вашего ответа, чтобы рассказать подробнее!


In [ ]:
user_message = "какой у вас выбор чаев и напитков?"
print(generate_bot(user_message))

У нас действительно широкий выбор фиточаев из диких трав, каждый из которых имеет свои уникальные свойства. Позвольте предложить вам рассмотреть следующие варианты:

1. **Фиточай из диких трав № 1 (Очищение и дренаж)** – отличный выбор для тех, кто хочет поддержать detox-процессы в организме.
2. **Фиточай из диких трав № 2 (Женская гармония)** – специально разработан для поддержания женского здоровья и гармонии.
3. **Фиточай из диких трав № 3 (Природный антистресс)** – поможет справиться с напряжением и стрессом, обеспечивая успокаивающий эффект.
4. **Фиточай из диких трав № 4 (Легкое дыхание)** – способствует улучшению дыхательной функции и поддерживает здоровье легких.
5. **Фиточай из диких трав № 5 (Комфортное пищеварение)** – поддерживает нормальную работу пищеварительной системы и помогает при дискомфорте.
6. **Фиточай из диких трав № 6 (Защита печени)** – направлен на поддержку здоровья печени и детоксикацию.
7. **Фиточай из диких трав № 7 (Легкость движений)** – поможет сохранит

In [ ]:
user_message = "что можно узнать про Углеводный контроль"
print(generate_bot(user_message))

Я благодарен вам за ваш выбор нашего продукта — Фиточая из диких трав № 9 (Углеводный контроль) из коллекции Baikal Tea Collection. Этот ароматный травяной чай содержит компоненты, такие как корень лопуха, трава люцерны и побеги черники, которые помогают мягко очищать организм от продуктов углеводного обмена. Он станет отличным решением для любителей сладкого и тех, кто хочет контролировать уровень сахара в крови.

**Как использовать:** Залейте один фильтр-пакет 200 мл кипятка, дайте настояться 10–15 минут и наслаждайтесь напитком во время любого приема пищи. Важно отметить, что настоящий вкус травяного купажа раскрывается через 5–10 минут после заваривания, а охлажденный чай сохраняет все свои полезные свойства.

**Состав:** Створки фасоли, побеги черники обыкновенной, корневища и корни элеутерококка, трава люцерны, листья шалфея, чага и корни лопуха.

Этот продукт отлично подходит для людей, стремящихся к поддержанию здоровья и нормализации углеводного обмена. Он доступен в упаковке 

In [ ]:
user_message = "Фиточай из диких трав № 5 цена"
#user_response_message = response.response_user_mesage(user_message=user_message, data=keys.all_values(data=logik_data))
#if_text.if_else(user_response_message)
#user_response_message
print(generate_bot(user_message))

Я благодарен вам за ваш выбор нашего продукта — Фиточай из диких трав № 1 (Очищение и дренаж) из коллекции Baikal Tea Collection. Этот травяной сбор, состоящий из курильского чая, сенны и клевера, подарит вам ощущение легкости и станет отличным спутником во время диеты.

Фиточай помогает вывести излишки жидкости и избавиться от накопленных токсинов, что особенно актуально для жителей больших городов. Каждый пакетик чая следует заваривать в 200 мл кипятка и настаивать 10–15 минут. Полезно знать, что охлажденный травяной чай сохраняет свои полезные свойства.

Цена на этот продукт составляет 270 ₽ за упаковку, в которой 30 фильтр-пакетов. В его состав входят листья сенны, кора крушины, листья крапивы, корень лопуха, листья смородины, цветки ромашки и курильский чай.

Если у вас возникли дополнительные вопросы или вы хотите приобрести этот фиточай, я с радостью помогу вам связаться с менеджером для оформления заказа.


In [ ]:
user_message = "Фиточай из диких трав № 7  состав и стоимость"
print(generate_bot(user_message))

Я благодарен вам за ваш выбор нашего продукта — Фиточай из диких трав № 1 (Очищение и дренаж) из коллекции Baikal Tea Collection. Это травяной сбор с курильским чаем, сенной и клевером, который подарит вам ощущение легкости и станет отличным спутником во время диеты. Очищающий купаж поможет вывести излишки жидкости и избавиться от накопленных токсинов, что особенно актуально для жителей больших городов.

Состав: 
- Листья сенны
- Кора крушины
- Листья крапивы
- Корень лопуха
- Листья смородины
- Цветки ромашки
- Курильский чай таежный
- Трава клевера

Как использовать:
1. Один фильтр-пакет залить 200 мл кипятка.
2. Дать настояться в течение 10–15 минут.
3. Принимать во время еды.

Полезно знать: настоящий вкус травяного купажа раскрывается через 5–10 минут после заваривания. Охлажденный травяной чай сохраняет свои полезные свойства.

Стоимость данного фиточая составляет 270 ₽ за упаковку, в которой 30 фильтр-пакетов.

Если у вас возникли дополнительные вопросы или вы хотите приобрести 

In [ ]:
user_message = "категория для сорта, какие в ней имеются товары?"
print(generate_bot(user_message))

Я ценю ваш выбор, и могу предложить определиться с категорией. Ранее мы с вами рассматривали такие варианты, как:

- Витамины и добавки
- Для спорта
- Здоровое питание
- Для поддержания красоты
- Парфюмерия
- Чаи и напитки
- Бальзамы
- Сопутствующие товары

Если вас интересует какая-то конкретная категория, я с удовольствием расскажу о товарах из нее!


In [ ]:
user_message = "что у вас есть в разделе Для спорта"
print(generate_bot(user_message))

Я вижу, что вас интересует категория для спорта. У нас есть несколько подкатегорий, которые могут вас заинтересовать. Позвольте представить их подробнее.

1. **Протеины** – отличный выбор для поддержания мышечной массы и восстановления после тренировок. У нас есть различные виды протеинов, включая сывороточный, казеиновый и растительный.

2. **Аминокислоты** – незаменимый элемент для спортсменов, помогающий в восстановлении и поддержании энергии во время тренировок. Мы предлагаем как BCAA, так и комплексные аминокислотные продукты.

3. **Витамины и микроэлементы** – важны для поддержания общего здоровья и повышения выносливости. Наши комплексы содержат все необходимые вещества для активного образа жизни.

4. **Продукты для суставов** – если вам важно поддерживать здоровье суставов, у нас есть специальные средства, которые помогают в этом, особенно для активных людей.

5. **Коллаген** – это отличный продукт для поддержания здоровья кожи и суставов. Он способствует восстановлению и улучш

In [ ]:
user_message = "Продукты для суставов"
print(generate_bot(user_message))

Выбор продуктов для поддержания здоровья суставов — это важный шаг, и из нашей подкатегории я мог бы вам порекомендовать несколько отличных вариантов. 

Во-первых, **Чистый рыбный коллаген (клубника-лайм) Fitness Catalyst**. Этот продукт содержит коллаген, который помогает поддерживать эластичность суставов и улучшает их функционирование, а также вкусный фруктовый привкус, что делает его приятным в употреблении.

Во-вторых, **Метилсульфонилметан** (МСМ) — это природное соединение, которое способствует снижению воспалительных процессов в суставах и помогает уменьшить болевые ощущения. Он также поддерживает здоровье соединительных тканей.

И, наконец, **Глюкозамин и хондроитин** — это два ключевых компонента, которые играют важную роль в восстановлении хрящевой ткани и обеспечении надлежащей подвижности суставов. Эти вещества помогают улучшить общую гибкость и снизить риск повреждений.

Я рекомендую рассмотреть этот список товаров, чтобы выбрать оптимальные решения для здоровья ваших сус

In [ ]:
user_message = "(клубника-лайм)"
print(generate_bot(user_message))

Я благодарен вам за ваш выбор нашего продукта «Арония и лютеин» из линии Essential Botanics. Этот растительный комплекс, основанный на лютеине, антоцианах черники и черноплодной рябины, обеспечивает надежную защиту глаз от негативного воздействия ультрафиолета и электронного излучения экранов мобильных гаджетов.

Рекомендуется принимать по 1-2 капсулы в день во время еды, а продолжительность приема составляет 1 месяц. В составе вы найдете порошок из рябины черноплодной, экстракт черники и лютеин, а также оболочку капсулы из говяжьего желатина. Продукт содержит активные компоненты: антоцианы (15 мг) и лютеин (2 мг), которые способствуют улучшению зрения и защите глаз.

Этот продукт стоит 550 рублей за упаковку из 30 капсул. Если у вас возникли дополнительные вопросы или вы хотите приобрести «Арония и лютеин», я с удовольствием помогу вам связаться с менеджером для оформления заказа.


In [ ]:
history

[('Добрый день! Я нейроконсультант компании Siberian Wellness, и с удовольствием расскажу вам о нашей компании.\n\nSiberian Wellness – это международная компания, которая предлагает инновационные продукты для здоровья, красоты и спорта. Мы гордимся 28-летней историей стабильного бизнеса, охватывающего более 60 стран, от Азии до США. В нашем ассортименте более 300 уникальных формул, которые получили более 50 наград за качество и эффективность.\n\nНаша компания основана на сибирских корнях, где уважение к традициям, науке и природе занимает центральное место. Мы производим натуральные продукты и эксклюзивную парфюмерию, используя богатство местной природы и достижения науки. Наши производственные комплексы расположены в России, Сербии, Италии и Узбекистане, что позволяет нам развивать специализированные линейки продуктов, учитывающие местные особенности.\n\nМы стремимся делать wellness-доступным для всех, и наше главное достижение – это доверие наших Бизнес-Партнеров и сотен тысяч Клиент

In [ ]:
extruder.one_price(art_name(user_response_message, prod, key_prod))

{'500589': {'sku': '500589',
  'name': 'Фиточай из диких трав № 9 (Углеводный контроль)',
  'shortName': 'Baikal Tea Collection',
  'description': 'Baikal Tea Collection',
  'shortDescription': 'Начните новый день с пользой! Чашка ароматного травяного чая на основе корня лопуха, травы люцерны и побегов черники поможет мягко очистить организм от продуктов углеводного обмена. Натуральный сбор – отличное решение для любителей сладкого и тех, кому важно держать уровень сахара под контролем.',
  'howToUse': '<p>1. Залейте один фильтр-пакет 200 мл кипятка.<br />2. Даете настояться в течение 10&ndash;15 минут.<br />3. Наслаждайтесь напитком во время любого приема пищи.</p>\n<p><strong>Полезно знать:</strong><br />Настоящий вкус травяного купажа раскрывается через 5&ndash;10 минут после заваривания.<br />Охлажденный травяной чай остается таким же полезным, как и сразу после заваривания.</p>',
  'contraindications': '',
  'productComponent': '<p><span class="">Створки фасоли, побеги черники обы

In [ ]:
bot=BotopenAI(client=client,
              model=model,
              messages=[{"role": "user", "content": prompt.prompt_base(level='приветствие', info_data=keys.keys_level(0), user_message=user_message)}],
              max_tokens=max_tokens, temperature=temperature)

In [ ]:
all_price=extruder.load_all_products()

In [ ]:
one_price=all_price[1]

In [ ]:
one_price

In [ ]:
product = Product(one_price['sku'],one_price['name'],one_price['shortName'],one_price['description'],one_price['shortDescription'],one_price['howToUse'],one_price['contraindications'],one_price['productComponent'],one_price['productComponentRaw'],one_price['options'],one_price['price'],one_price['isAction'],one_price['categories'],one_price['topCategory'],one_price['thumbnails'])

In [ ]:
bot.used_bot()

"Здравствуйте! Я нейроконсультант компании 'Siberian Wellness'. Рад вас приветствовать! \n\nВы интересуетесь продуктами для улучшения работы мозга. К сожалению, в нашем ассортименте нет отдельной категории, посвященной именно этой теме. Однако, многие из наших продуктов могут поддерживать здоровье мозга и улучшать когнитивные функции.\n\nЕсли вас интересуют витамины и добавки, то я рекомендую обратить внимание на наши комплексы, которые поддерживают общее здоровье и активность. Также в категории «для спорта» можно найти продукты, способствующие лучшей концентрации и выносливости.\n\nЕсли вы хотите, я могу рассказать подробнее о каком-либо конкретном направлении, например, о витаминах и добавках или других продуктах. Какой из этих вариантов вас интересует больше?"

In [ ]:
keys.keys_level(dict_level=0, data=category_article)

['витамины и добавки',
 'для спорта',
 'здоровое питание',
 'для поддержания красоты',
 'парфюмерия',
 'чаи и напитки',
 'бальзамы',
 'сопутствующие товары']

In [ ]:
x=keys.find_values( target_key='когнитивное и психоэмоциональное здоровье')

In [ ]:
y=keys.find_values( target_key='успокоительные средства', data=x[0])

In [ ]:
keys.keys_level(dict_level=0, data=x[0])

['поддержка мозга', 'успокоительные средства']

In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

# Загрузка модели и токенизатора
model = AutoModel.from_pretrained('nvidia/NV-Embed-v2', trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained('nvidia/NV-Embed-v2')

# Список товаров
products = [
    "Смартфон Samsung Galaxy S21",
    "Ноутбук Apple MacBook Air",
    "Наушники Sony WH-1000XM4",
    "Умные часы Garmin Forerunner 245"
]

# Запрос для поиска
query = "лучшие наушники для музыки"

# Векторизация запроса и товаров
query_embedding = model.encode([query], instruction="Поиск товаров по ключевым словам", max_length=32768)
product_embeddings = model.encode(products, instruction="", max_length=32768)

# Нормализация векторов
query_embedding = torch.nn.functional.normalize(query_embedding, p=2, dim=1)
product_embeddings = torch.nn.functional.normalize(product_embeddings, p=2, dim=1)

# Вычисление схожести
scores = (query_embedding @ product_embeddings.T) * 100

# Вывод результатов
sorted_indices = scores[0].argsort(descending=True)
for index in sorted_indices:
    print(f"Товар: {products[index]}, Схожесть: {scores[0][index].item()}")

model.safetensors.index.json:   0%|          | 0.00/28.2k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/789M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/997 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
y

[['501291',
  '501431',
  '501186',
  '500931',
  '500657',
  '500583',
  '400264',
  '501596',
  '501591']]

In [ ]:
keys.find_values( 'успокоительные средства')

[['501291',
  '501431',
  '501186',
  '500931',
  '500657',
  '500583',
  '400264',
  '501596',
  '501591']]

In [ ]:
keys.all_keys()

In [ ]:
keys.all_values()

In [ ]:
keys.find_deepest_key_level('витамины и добавки')

0

In [ ]:
keys.find_key_level('успокоительные средства')

2

In [ ]:
keys.get_keys_in_category_at_level(category='иммунитет и защита организма', target_level=0)

[]

In [ ]:
article=keys.find_values('успокоительные средства')
article[0]

['501291',
 '501431',
 '501186',
 '500931',
 '500657',
 '500583',
 '400264',
 '501596',
 '501591']

In [ ]:
extruder.one_price(article[0][3])

In [ ]:
one_price

In [ ]:
product.get_price()

{'value': 1900,
 'currency': {'externalIso': 'RUB',
  'logo': '₽',
  'decimals': 2,
  'decimalPoint': '.',
  'thousandSeparator': ' ',
  'isPrefixed': False,
  'roundRules': 0}}

In [ ]:
product.get_name()

'RELAX Box / Защита от стресса'

In [ ]:
user_response_data=[]

In [ ]:
keys.keys_level(dict_level=1, data=category_article['витамины и добавки'])

['витамины и добавки',
 'для спорта',
 'здоровое питание',
 'для поддержания красоты',
 'парфюмерия',
 'чаи и напитки',
 'бальзамы',
 'сопутствующие товары']

In [ ]:
user_message = "есть ли у вас витаминные комплексы"
user_response = response.response_user_mesage(user_message=user_message, data=keys.keys_level(dict_level=0))
level = keys.find_deepest_key_level(user_response)
print(level, user_response, keys.keys_level(dict_level=0, data=category_article[user_response]))

0 витамины и добавки ['иммунитет и защита организма', 'поддержка органов и систем', 'здоровье суставов и костей', 'когнитивное и психоэмоциональное здоровье', 'красота и здоровье женщин', 'общее здоровье и профилактика']


In [ ]:
job_dikt(category_article, user_response)

['иммунитет и защита организма',
 'поддержка органов и систем',
 'здоровье суставов и костей',
 'когнитивное и психоэмоциональное здоровье',
 'красота и здоровье женщин',
 'общее здоровье и профилактика']

In [ ]:
user_message = "интересно чтонибудь для поддержки органов и систем"

In [ ]:
user_message = "меня интересуют товары для спорта"
user_response = response.response_user_mesage(user_message=user_message, data=next_level)
level = keys.find_deepest_key_level(user_response)
print(level, user_response)

2 другие товары


In [ ]:
next_level = keys.keys_level(dict_level=level)
next_level

['витамины и добавки',
 'для спорта',
 'здоровое питание',
 'для поддержания красоты',
 'парфюмерия',
 'чаи и напитки',
 'бальзамы',
 'сопутствующие товары']

In [ ]:
user_message = "У вас неплохой раздел с чаем"
response_user_text=response.response_user_mesage(user_message, category_urls)
if response_user_text==response.key_load(category_urls)[5]:
  if len(category_urls[response_user_text])<2:
    print(f'из категории {response_user_text}, могу вам порекомендовать {response.key_load(category_urls[response_user_text][response_user_text])}')
    if len(category_urls[response_user_text])<2:
      print(f'из категории {response_user_text}, могу вам порекомендовать {response.key_load(category_urls[response_user_text][response_user_text])}')
  else:
    print(f'из категории {response_user_text}, могу вам порекомендовать {response.key_load(category_urls[response_user_text])}')

из категории чаи и напитки, могу вам порекомендовать ['чаи и напитки']
из категории чаи и напитки, могу вам порекомендовать ['чаи и напитки']


In [ ]:
response.key_load(category_urls)[2]

'здоровое питание'

In [ ]:
print(f'из категории {response_user_text}, могу вам порекомендовать {response.key_load(category_urls[response_user_text])}')

из категории чаи и напитки, могу вам порекомендовать ['чаи и напитки']


In [ ]:
category_urls['витамины и добавки']['иммунитет и защита организма']

{'сильный иммунитет': 'https://ru.siberianhealth.com/ru/shop/catalog/vitaminy-i-dobavki-1/f/silnyy-immunitet-15/',
 'антиоксиданты': 'https://ru.siberianhealth.com/ru/shop/catalog/vitaminy-i-dobavki-1/f/antioksidanty-27/',
 'антипаразитарная защита': 'https://ru.siberianhealth.com/ru/shop/catalog/vitaminy-i-dobavki-1/f/antiparazitarnaya-zashchita-24/',
 'профилактика аллергии': 'https://ru.siberianhealth.com/ru/shop/catalog/vitaminy-i-dobavki-1/f/profilaktika-allergii-13/',
 'профилактика диабета': 'https://ru.siberianhealth.com/ru/shop/catalog/vitaminy-i-dobavki-1/f/profilaktika-diabeta-14/'}

Это тестовое поле блокнота, здесь старые версии и тестовые отработки

In [ ]:
product = Extruder(headers, category_urls, 'medium', 'https://ru.siberianhealth.com/ru/')

In [ ]:
response = requests.get('https://ru.siberianhealth.com/ru/', headers)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
soup

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import pickle
# saving
with open('/content/drive/MyDrive/famili/all_product.pickle', 'wb') as handle:
    pickle.dump(all_product, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
all_prod

In [ ]:
testclass = Extruder(headers, category_urls, 'medium')

In [ ]:
print(testclass.one_category('https://ru.siberianhealth.com/ru/shop/catalog/parfyumeriya-212/'))

({'108189': 'Episode 26, парфюмерная вода, 1,5 мл', '425139': 'Episode 26, парфюмерная вода', '108188': 'Episode 23, парфюмерная вода, 1,5 мл', '425137': 'Episode 23, парфюмерная вода', '425175': 'Nuage Gardenia, парфюмерная вода', '108214': 'Парфюмерная вода Arc-en-Ciel, 1.5 мл', '108199': 'Arc-en-Ciel White, парфюмерная вода, 1,5 мл', '108200': 'Arc-en-Ciel Pink, парфюмерная вода, 1.5 мл', '108201': 'Arc-en-Ciel Red, парфюмерная вода, 1,5 мл', '425250': 'Arc-en-Ciel Red, парфюмерная вода, 20 мл', '425248': 'Arc-en-Ciel Pink, парфюмерная вода, 20 мл', '425246': 'Arc-en-Ciel Emerald, парфюмерная вода, 20 мл', '427757': 'Подарочный набор изысканных ароматов «Секрет абсолютной женственности»', '427745': 'Парфюмированный мист «Лесной орех & Бобы тонка» для ароматизации помещений и текстиля', '108212': 'Arc-en-ciel Bloom, парфюмерная вода, 1,5 мл', '427653': 'Парфюмерная вода Masala tea & Tobacco (Чай масала и Табак), 50 мл', '427652': 'Парфюмерная вода Blue Matcha & Vetiver (Голубая матча

In [ ]:
print(id_1234.get_structure())

мыльная основа на растительном сырье, ароматические масла, натуральный краситель бета каротин


In [ ]:
product.price = 500
print(product.price)


475.0


In [ ]:
data=['спорт','здоровье','питание','красота']
text= 'Добрый день! Проконсультируйте по товарам?'
data_2=['питание','мышечьная масса','энергия','здоровые суставы']
text_2='Есть ли что нибудь для суставов в категории спорта?'
data_3=['Мазь живокост','Экстракт природного кальция','Минеральный комплекс для сустаов','Коктель с природным коллагеном для здоровья суставов']
text_3='Категория здоровые суставы мне как раз интересна'
prompt_1=Prompt()
print(prompt_1.prompt_base('приветствие', data, text))
prompt_2=Prompt()
print(prompt_2.prompt_base('подкат', data_2, text_2))

Ты менеджер компании 'Siberian Wellnes', твоя задача проконсультировать пользователя по продуктам компании.
        Представься как нейроконсультант компании 'Siberian Wellness', спроси у пользователя в какой категории он 
        рассматривает приобретение продукта, вот список категорий ['спорт', 'здоровье', 'питание', 'красота'], вот запрос пользователя: Добрый день! Проконсультируйте по товарам?. будь вежливым и вариативным, не придумывай факты, придерживайся предоставленной информации,
         старайся уложиться в 100 токенов
Ты менеджер компании 'Siberian Wellnes', твоя задача проконсультировать пользователя по продуктам компании.
        Пользователь определился с категорией Есть ли что нибудь для суставов в категории спорта?, примени подобный оборот вариативно, [я вижу вас интересует эта категория Есть ли что нибудь для суставов в категории спорта?], 
        предложи расмотреть подкатегории в этом направлении ['питание', 'мышечьная масса', 'энергия', 'здоровые суставы'], предс

In [ ]:


bot=BotopenAI(client, model, messages, max_tokens, temperature, response_data)
bot2=BotopenAI(client, model, messages_2, 450, temperature, response_data)
bot3=BotopenAI(client, model, messages_3, 450, temperature, response_data)

In [ ]:
print(bot.generate_support_response())
print(bot2.generate_support_response())
print(bot3.generate_support_response())

Добрый день! Я нейроконсультант компании 'Siberian Wellness'. С радостью помогу вам с выбором продуктов. В какой категории вы рассматриваете приобретение товара? У нас есть замечательные предложения в категориях: 'спорт', 'здоровье', 'питание' и 'красота'.
Здравствуйте! Я вижу, что вас интересует категория, связанная с суставами в контексте спорта. Это замечательно, так как забота о суставах очень важна для активного образа жизни и физической активности.

В нашем ассортименте есть несколько подкатегорий, которые могут помочь вам в этом направлении. Позвольте мне представить их:

1. **Питание**: В этой подкатегории вы найдете продукты, которые способствуют улучшению общего состояния суставов и поддержанию их здоровья. Это важно для профилактики травм и восстановления после нагрузок.

2. **Мышечная масса**: Здесь представлены добавки и продукты, которые помогают в увеличении мышечной массы. Здоровые мышцы поддерживают суставы, поэтому это направление также важно для вашей физической акти